In [6]:
from brian2 import *
import numpy as np
import logging
import warnings



from brian2 import prefs, set_device

# Tell Brian2 to use the Cython code generator:
prefs.codegen.target = 'cython'

# Optionally compile but keep Python interface:
set_device('runtime')  # default; compiles operations to .so but stays in Python process





# suppress overflow warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)
numpy.seterr(over='ignore', under='ignore')
logging.getLogger('brian2').setLevel(logging.ERROR)

# ----------------------------------------------------------------------------
# Spike timing and derivative

start_scope()
defaultclock.dt = 0.001*ms

@implementation('numpy', discard_units=True)
@check_units(w=1, global_clock=1, layer=1, sum=1, spikes_received=1, result=1)
def spike_timing(w, global_clock, layer, sum, spikes_received):
    x = global_clock % 1
    if w >= 0:
        return np.power(x, (1 - w), where=(x>0), out=np.zeros_like(x))
    else:
        return 1 - np.power((1 - x), (1 + w), where=(x<1), out=np.ones_like(x))

@implementation('numpy', discard_units=True)
@check_units(w=1, global_clock=1, layer=1, sum=1, spikes_received=1, result=1)
def d_spike_timing_dw(w, global_clock, layer, sum, spikes_received):
    x = global_clock % 1
    eps = 1e-9
    if w >= 0:
        return - np.power(x, (1 - w), where=(x>0), out=np.zeros_like(x)) * np.log(x + eps)
    else:
        return - np.power((1 - x), (1 + w), where=(x<1), out=np.ones_like(x)) * np.log(1 - x + eps)

# ----------------------------------------------------------------------------
# Forward pass: 4->10->3 using two-stage mini_urd

def layer_forward(inputs, W, layer_idx):
    """
    inputs: array of spike times (ms) from previous layer (shape: n_in,)
    W: weight matrix shape (n_in+1, n_out)  ← note the extra bias row
    layer_idx: integer layer number
    returns: array of output spike times (ms)
    """
    # 1) augment inputs with bias spike @ t=0
    bias_time = 0.0
    aug_inputs = np.concatenate((inputs, [bias_time]))  # shape (n_in+1,)

    n_in_plus_bias, n_out = W.shape
    assert aug_inputs.size == n_in_plus_bias

    out_times = []
    for j in range(n_out):
        start_scope()
        defaultclock.dt = 0.001*ms

        # single post‐synaptic neuron
        G = NeuronGroup(1, '''
            v : 1
            sum : 1
            sr : 1
            scheduled_time : second
            global_clock : 1
        ''', threshold='v>1', reset='v=0', method='exact')

        # init
        G.v = G.sum = G.sr = 0
        G.global_clock = 0
        G.scheduled_time = 1e9*second

        # stim: now includes bias spike at t=0
        stim = SpikeGeneratorGroup(n_in_plus_bias,
                                   indices=list(range(n_in_plus_bias)),
                                   times=aug_inputs*ms)

        S = Synapses(stim, G, '''w:1
            layer:1''',
            on_pre='''
            sr += 1
            sum += spike_timing(w, global_clock, layer, sum, sr)
            scheduled_time = (sum/sr + layer)*ms
        ''')
        S.connect(True)
        S.w = W[:, j]
        S.layer = layer_idx

        G.run_regularly('''
            v = int(abs(t - scheduled_time) < 0.0005*ms) * 1.2
            global_clock += 0.001
        ''', dt=0.001*ms)

        mon = SpikeMonitor(G)
        run(5*ms)

        ts = mon.spike_trains()[0]
        t0 = float(ts[0]/ms) if len(ts)>0 else float(5.0)
        out_times.append(t0)

    return np.array(out_times)


# ----------------------------------------------------------------------------
# Training loop with backprop for 4-10-3
def train_snn_backprop(
    X, Y,                # lists of input arrays (4,) and target (3,)
    W1_init, W2_init,
    epochs=10, lr=0.1,
    max_grad=20.0, w_min=-20.0, w_max=20.0,
    non_target_time=2.0,
    λ=0.5                # non-target penalty weight
):
    """
    Trains a 4→10→3 spiking network with:
      • batched gradient updates
      • boosted hidden-layer learning rate
      • separate gradient clipping per layer
      • classical momentum smoothing
    """
    # Initialize weights
    W1 = W1_init.copy()      # shape (5,10) including bias row
    W2 = W2_init.copy()      # shape (11,3) including bias row

    # Momentum buffers
    beta = 0.9
    vW1 = np.zeros_like(W1)
    vW2 = np.zeros_like(W2)

    layer1_idx, layer2_idx = 1, 2
    N = len(X)

    for ep in range(epochs):
        # Accumulators for this epoch
        acc_dW1 = np.zeros_like(W1)
        acc_dW2 = np.zeros_like(W2)
        epoch_loss = 0.0

        for xi, yi in zip(X, Y):
            # — Forward pass —
            h_times = layer_forward(xi, W1, layer1_idx)
            o_times = layer_forward(h_times, W2, layer2_idx)

            # — Separation loss —
            target_idx = np.argmax(yi)
            L_target = 0.5 * (o_times[target_idx] - yi[target_idx])**2
            non_ids = [j for j in range(len(o_times)) if j != target_idx]
            L_non = 0.5 * λ * sum((o_times[j] - non_target_time)**2 for j in non_ids)
            L = L_target + L_non
            epoch_loss += L

            # — Gradients for W2 —
            delta_o = np.zeros_like(o_times)
            delta_o[target_idx] = (o_times[target_idx] - yi[target_idx])
            for j in non_ids:
                delta_o[j] = λ * (o_times[j] - non_target_time)

            aug_h = np.concatenate((h_times, [0.0]))
            dW2 = np.zeros_like(W2)
            for k in range(W2.shape[0]):
                for j in range(W2.shape[1]):
                    dW2[k, j] = delta_o[j] * d_spike_timing_dw(
                        W2[k, j], aug_h[k], layer2_idx, 0, 1)

            # — Backprop into hidden & gradients for W1 —
            delta_h = np.zeros_like(h_times)
            for k in range(len(h_times)):
                for j in range(W2.shape[1]):
                    dt_dw_output = d_spike_timing_dw(W2[k, j], aug_h[k], layer2_idx, 0, 1)
                    delta_h[k] += delta_o[j] * dt_dw_output  # Remove the W2[k,j] multiplication

            aug_xi = np.concatenate((xi, [0.0]))
            dW1 = np.zeros_like(W1)
            for i in range(W1.shape[0]):
                for k in range(W1.shape[1]):
                    dW1[i, k] = delta_h[k] * d_spike_timing_dw(
                        W1[i, k], aug_xi[i], layer1_idx, 0, 1)

            # — Accumulate —
            acc_dW1 += dW1
            acc_dW2 += dW2

        # — Average & clip gradients —
        acc_dW1 /= N
        acc_dW2 /= N

        # Boost hidden-layer rate
        lr1 = 5 * lr

        # Separate clipping thresholds
        g1 = np.clip(acc_dW1, -max_grad*5, max_grad*5)
        g2 = np.clip(acc_dW2, -max_grad,   max_grad)

        # — Momentum updates —
        vW1 = beta * vW1 + (1 - beta) * g1
        vW2 = beta * vW2 + (1 - beta) * g2

        # — Apply weight updates & clamp —
        W1 = np.clip(W1 - lr1 * vW1, w_min, w_max)
        W2 = np.clip(W2 - lr  * vW2, w_min, w_max)

        print(f"Epoch {ep+1}/{epochs} — avg loss={epoch_loss/N:.4f}")
        print(f"             ‖W1‖={np.linalg.norm(W1):.3f}, ‖W2‖={np.linalg.norm(W2):.3f}\n")

    return W1, W2






            
            # # print changes before applying them
            # print("\nWeight changes for this sample:")
            # for i in range(W1.shape[0]):
            #     for k in range(W1.shape[1]):
            #         change = -lr * dW1[i,k]
            #         print(f"  W1[{i},{k}] change: {change:+.5f}")
            # for k in range(W2.shape[0]):
            #     for j in range(W2.shape[1]):
            #         change = -lr * dW2[k,j]
            #         print(f"  W2[{k},{j}] change: {change:+.5f}")
            


if __name__ == "__main__":
    # example usage with fixed input/target pairs
    # 4 inputs per sample, constant across 8 samples
    x0 = np.array([0.8, 0.7, 0.3, 0.6])
    x1 = np.array([0.9, 0.5, 0.5, 0.2])
    X = [x0 if i % 2 == 0 else x1 for i in range(8)]
    # 3-targets (network outputs 3 values): use desired spike times [2.1, 2.0, 1.0]
    y0 = np.array([2.95, 2.05, 2.05])
    y1 = np.array([2.05, 2.05, 2.95])
    Y = [y0 if i % 2 == 0 else y1 for i in range(8)]
    # X= []
    # Y = []
    # for _ in range(10):
    #     X.append(x0 + np.random.randn(4)*0.02);  Y.append(y0)
    #     X.append(x1 + np.random.randn(4)*0.02);  Y.append(y1)
    

    
    W1_0 = np.array([[0.21958991, 0.16223261, 0.02545166, 0.18849804, 0.09521701, 0.22744421, 0.05556097, 0.33130229, 0.03974721, 0.1968464],
                [0.41958955, 0.47541312, 0.22287581, 0.69627866, 0.83639384, 0.79597959, 0.15029805, 0.126486, 0.18285382, 0.07470098],
                [0.69559509, 0.41228614, 0.06028855, 0.51098037, 0.33730611, 1.17605488, 0.15405119, 0.28079173, 0.17365651, 0.23041775],
                [0.79721356, 0.82210554, 0.15028745, 1.09421856, 0.68280376, 1.07577422, 0.16962136, 0.23838796, 0.0735181, 0.1719861],
                [0.0631449, 0.10618091, 0.05791614, 0.0260418, -0.01797577, -0.1209534, 0.18702474, -0.01662061, -0.0683026, 0.05468931]])
        #W1_0 = [[ 0.21958991,  0.16223261,  0.02545166  0.18849804  0.09521701  0.22744421, 0.05556097  0.33130229  0.03974721  0.1968464 ], [ 0.41958955  0.47541312  0.22287581  0.69627866  0.83639384  0.79597959, 0.15029805  0.126486    0.18285382  0.07470098], [ 0.69559509  0.41228614  0.06028855  0.51098037  0.33730611  1.17605488, 0.15405119  0.28079173  0.17365651  0.23041775], [ 0.79721356  0.82210554  0.15028745  1.09421856  0.68280376  1.07577422, 0.16962136  0.23838796  0.0735181   0.1719861 ], [ 0.0631449   0.10618091  0.05791614  0.0260418  -0.01797577 -0.1209534, 0.18702474 -0.01662061 -0.0683026   0.05468931]]
        
    
        
        #np.random.randn(4+1, 10)*0.1   # +1 for bias

    
    W2_0 = np.array([[ 0.38843549, -1.10085101,  0.38776897],
                [ 0.35841177, -1.06985881,  0.40542767],
                [ 0.40732835, -0.63317637,  0.59202003],
                [ 0.32589618, -0.9691458,   0.48481597],
                [ 0.26060079, -0.85267046,  0.75057083],
                [ 0.31287437, -1.34806606,  0.44407244],
                [ 0.1996638,  -0.65507816,  0.27473825],
                [ 0.40169137, -0.9979045,   0.09884702],
                [ 0.37579471, -0.62826323,  0.58035222],
                [ 0.2191151,  -0.84980247,  0.14767976],
                [ 0.11199583, -0.16498428,  0.00871235]])
    
    # will print out last times so DO NOT run the same the same expermeent to have a differnt outcoem


    # train
    W1_tr, W2_tr = train_snn_backprop(X, Y, W1_0, W2_0,
                                      epochs=30, lr=0.5)
    print("Trained W1:", W1_tr)
    print("Trained W2:", W2_tr) 
    print("Hidden times for x0:", layer_forward(x0, W1_tr, 1))
    print("Hidden times for x1:", layer_forward(x1, W1_tr, 1))

    # ── Now test on the same two patterns ──
    print("\n=== Test predictions ===")
    for xi, yi in zip(X, Y):
        # call layer_forward(positionally) rather than with layer1_idx=
        h_times = layer_forward(xi, W1_tr, 1)
        o_times = layer_forward(h_times, W2_tr, 2)

        pred_class = np.argmax(o_times)  # changed to argmin WHY???
        true_class = np.argmax(yi)

        print(f"Input: {xi}")
        print(f" Spike times: {o_times}")
        print(f" Predicted class: {pred_class}, True class: {true_class}\n")




WARNING    C:\Users\irtho\AppData\Local\Temp\ipykernel_21960\3780920302.py:161: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  L_non = 0.5 * λ * sum((o_times[j] - non_target_time)**2 for j in non_ids)
 [py.warnings]


Epoch 1/30 — avg loss=0.1507
             ‖W1‖=3.104, ‖W2‖=3.468

Epoch 2/30 — avg loss=0.1507
             ‖W1‖=3.106, ‖W2‖=3.467

Epoch 3/30 — avg loss=0.1506
             ‖W1‖=3.110, ‖W2‖=3.465

Epoch 4/30 — avg loss=0.1506
             ‖W1‖=3.115, ‖W2‖=3.462

Epoch 5/30 — avg loss=0.1509
             ‖W1‖=3.122, ‖W2‖=3.459



KeyboardInterrupt: 

In [7]:
from brian2 import *
import numpy as np
import logging
import warnings

from brian2 import prefs, set_device

# Tell Brian2 to use the Cython code generator:
prefs.codegen.target = 'cython'
set_device('runtime')

# suppress overflow warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)
numpy.seterr(over='ignore', under='ignore')
logging.getLogger('brian2').setLevel(logging.ERROR)

# ----------------------------------------------------------------------------
# Spike timing and derivative with improved numerical stability

start_scope()
defaultclock.dt = 0.001*ms

@implementation('numpy', discard_units=True)
@check_units(w=1, global_clock=1, layer=1, sum=1, spikes_received=1, result=1)
def spike_timing(w, global_clock, layer, sum, spikes_received):
    x = np.clip(global_clock % 1, 1e-8, 1-1e-8)  # Avoid boundary issues
    if w >= 0:
        return np.power(x, (1 - w))
    else:
        return 1 - np.power((1 - x), (1 + w))

@implementation('numpy', discard_units=True)
@check_units(w=1, global_clock=1, layer=1, sum=1, spikes_received=1, result=1)
def d_spike_timing_dw(w, global_clock, layer, sum, spikes_received):
    x = np.clip(global_clock % 1, 1e-8, 1-1e-8)  # Avoid boundary issues
    eps = 1e-9
    if w >= 0:
        log_x = np.log(np.maximum(x, eps))
        return -np.power(x, (1 - w)) * log_x
    else:
        log_1_minus_x = np.log(np.maximum(1 - x, eps))
        return -np.power((1 - x), (1 + w)) * log_1_minus_x

# ----------------------------------------------------------------------------
# Forward pass: 4->10->3 using two-stage mini_urd

def layer_forward(inputs, W, layer_idx):
    """
    inputs: array of spike times (ms) from previous layer (shape: n_in,)
    W: weight matrix shape (n_in+1, n_out)  ← note the extra bias row
    layer_idx: integer layer number
    returns: array of output spike times (ms)
    """
    # 1) augment inputs with bias spike @ t=0
    bias_time = 0.0
    aug_inputs = np.concatenate((inputs, [bias_time]))  # shape (n_in+1,)

    n_in_plus_bias, n_out = W.shape
    assert aug_inputs.size == n_in_plus_bias

    out_times = []
    for j in range(n_out):
        start_scope()
        defaultclock.dt = 0.001*ms

        # single post‐synaptic neuron
        G = NeuronGroup(1, '''
            v : 1
            sum : 1
            sr : 1
            scheduled_time : second
            global_clock : 1
        ''', threshold='v>1', reset='v=0', method='exact')

        # init
        G.v = G.sum = G.sr = 0
        G.global_clock = 0
        G.scheduled_time = 1e9*second

        # stim: now includes bias spike at t=0
        stim = SpikeGeneratorGroup(n_in_plus_bias,
                                   indices=list(range(n_in_plus_bias)),
                                   times=aug_inputs*ms)

        S = Synapses(stim, G, '''w:1
            layer:1''',
            on_pre='''
            sr += 1
            sum += spike_timing(w, global_clock, layer, sum, sr)
            scheduled_time = (sum/sr + layer)*ms
        ''')
        S.connect(True)
        S.w = W[:, j]
        S.layer = layer_idx

        G.run_regularly('''
            v = int(abs(t - scheduled_time) < 0.0005*ms) * 1.2
            global_clock += 0.001
        ''', dt=0.001*ms)

        mon = SpikeMonitor(G)
        run(5*ms)

        ts = mon.spike_trains()[0]
        t0 = float(ts[0]/ms) if len(ts)>0 else float(5.0)
        out_times.append(t0)

    return np.array(out_times)

# ----------------------------------------------------------------------------
# Training loop with corrected backprop for 4-10-3

def train_snn_backprop(
    X, Y,                # lists of input arrays (4,) and target (3,)
    W1_init, W2_init,
    epochs=10, lr=0.05,   # Reduced learning rate
    max_grad=5.0,         # Reduced gradient clipping
    w_min=-10.0, w_max=10.0,  # Reduced weight bounds
    non_target_time=2.0,  # Non-target neurons should spike at 2.0ms
    λ=0.2                 # Reduced penalty weight
):
    """
    Trains a 4→10→3 spiking network where:
    - Target neuron should spike at specified time (2.95ms)
    - Non-target neurons should spike at 2.0ms
    - Classification uses argmax (latest spike wins)
    """
    # Initialize weights
    W1 = W1_init.copy()      # shape (5,10) including bias row
    W2 = W2_init.copy()      # shape (11,3) including bias row

    # Momentum buffers
    beta = 0.9
    vW1 = np.zeros_like(W1)
    vW2 = np.zeros_like(W2)

    layer1_idx, layer2_idx = 1, 2
    N = len(X)

    for ep in range(epochs):
        # Accumulators for this epoch
        acc_dW1 = np.zeros_like(W1)
        acc_dW2 = np.zeros_like(W2)
        epoch_loss = 0.0

        for xi, yi in zip(X, Y):
            # — Forward pass —
            h_times = layer_forward(xi, W1, layer1_idx)
            o_times = layer_forward(h_times, W2, layer2_idx)

            # — Loss computation —
            target_idx = np.argmax(yi)
            L_target = 0.5 * (o_times[target_idx] - yi[target_idx])**2
            
            # Non-target loss: push other neurons to fire at 2.0ms
            non_ids = [j for j in range(len(o_times)) if j != target_idx]
            L_non = 0.5 * λ * sum((o_times[j] - non_target_time)**2 for j in non_ids)
            L = L_target + L_non
            epoch_loss += L

            # — Output layer gradients (W2) —
            delta_o = np.zeros_like(o_times)
            delta_o[target_idx] = (o_times[target_idx] - yi[target_idx])
            for j in non_ids:
                delta_o[j] = λ * (o_times[j] - non_target_time)

            # Compute dW2 gradients
            aug_h = np.concatenate((h_times, [0.0]))  # Add bias
            dW2 = np.zeros_like(W2)
            for k in range(W2.shape[0]):
                for j in range(W2.shape[1]):
                    if aug_h[k] > 0:  # Only compute if input exists
                        dt_dw = d_spike_timing_dw(W2[k, j], aug_h[k], layer2_idx, 0, 1)
                        dW2[k, j] = delta_o[j] * dt_dw

            # — Hidden layer gradients (backprop) —
            delta_h = np.zeros_like(h_times)
            for k in range(len(h_times)):
                for j in range(W2.shape[1]):
                    if aug_h[k] > 0:  # Only backprop if hidden neuron fired
                        # Chain rule: dL/dh_k = sum_j (dL/do_j * do_j/dh_k)
                        # do_j/dh_k = sum_i (do_j/dw2_ij * dw2_ij/dh_k)
                        # Since w2_ij is independent of h_k, we need do_j/dh_k directly
                        
                        # For spike timing, output depends on input time through the spike_timing function
                        # This is a simplified approximation - in practice, this relationship is complex
                        dt_dh = d_spike_timing_dw(W2[k, j], aug_h[k], layer2_idx, 0, 1)
                        delta_h[k] += delta_o[j] * dt_dh

            # Compute dW1 gradients
            aug_xi = np.concatenate((xi, [0.0]))  # Add bias
            dW1 = np.zeros_like(W1)
            for i in range(W1.shape[0]):
                for k in range(W1.shape[1]):
                    if aug_xi[i] > 0:  # Only compute if input exists
                        dt_dw = d_spike_timing_dw(W1[i, k], aug_xi[i], layer1_idx, 0, 1)
                        dW1[i, k] = delta_h[k] * dt_dw

            # — Accumulate gradients —
            acc_dW1 += dW1
            acc_dW2 += dW2

        # — Average gradients over batch —
        acc_dW1 /= N
        acc_dW2 /= N

        # — Clip gradients —
        acc_dW1 = np.clip(acc_dW1, -max_grad, max_grad)
        acc_dW2 = np.clip(acc_dW2, -max_grad, max_grad)

        # — Momentum updates —
        vW1 = beta * vW1 + (1 - beta) * acc_dW1
        vW2 = beta * vW2 + (1 - beta) * acc_dW2

        # — Apply weight updates & clamp —
        W1 = np.clip(W1 - lr * vW1, w_min, w_max)
        W2 = np.clip(W2 - lr * vW2, w_min, w_max)

        # — Progress report —
        if ep % 5 == 0 or ep == epochs - 1:
            print(f"Epoch {ep+1}/{epochs} — avg loss={epoch_loss/N:.4f}")
            print(f"             ‖W1‖={np.linalg.norm(W1):.3f}, ‖W2‖={np.linalg.norm(W2):.3f}")
            
            # Quick accuracy check
            correct = 0
            for xi, yi in zip(X, Y):
                h_times = layer_forward(xi, W1, layer1_idx)
                o_times = layer_forward(h_times, W2, layer2_idx)
                pred_class = np.argmax(o_times)
                true_class = np.argmax(yi)
                if pred_class == true_class:
                    correct += 1
            accuracy = correct / len(X)
            print(f"             Accuracy: {accuracy:.3f}")
            print()

    return W1, W2

if __name__ == "__main__":
    # Example usage with fixed input/target pairs
    x0 = np.array([0.8, 0.7, 0.3, 0.6])
    x1 = np.array([0.9, 0.5, 0.5, 0.2])
    X = [x0 if i % 2 == 0 else x1 for i in range(8)]
    
    # Target spike times: larger value = correct class
    # Class 0: neuron 0 should spike at 2.95ms, others at 2.0ms
    # Class 2: neuron 2 should spike at 2.95ms, others at 2.0ms
    y0 = np.array([2.95, 2.0, 2.0])   # Class 0 target
    y1 = np.array([2.0, 2.0, 2.95])   # Class 2 target
    Y = [y0 if i % 2 == 0 else y1 for i in range(8)]
    
    # Smaller, more stable weight initialization
    np.random.seed(42)  # For reproducibility
    W1_0 = np.random.randn(5, 10) * 0.1   # Smaller initial weights
    W2_0 = np.random.randn(11, 3) * 0.1
    
    print("=== Training ===")
    # Train with more conservative parameters
    W1_tr, W2_tr = train_snn_backprop(X, Y, W1_0, W2_0,
                                      epochs=50, lr=0.1)
    
    print("\n=== Final Test Results ===")
    print("Input pattern -> Output times -> Predicted class (True class)")
    print("-" * 60)
    
    for i, (xi, yi) in enumerate(zip(X, Y)):
        h_times = layer_forward(xi, W1_tr, 1)
        o_times = layer_forward(h_times, W2_tr, 2)
        
        pred_class = np.argmax(o_times)
        true_class = np.argmax(yi)
        
        print(f"Sample {i+1}: {xi}")
        print(f"  Output times: [{o_times[0]:.3f}, {o_times[1]:.3f}, {o_times[2]:.3f}]")
        print(f"  Predicted: {pred_class}, True: {true_class} {'✓' if pred_class == true_class else '✗'}")
        print()
    
    # Calculate final accuracy
    correct = 0
    for xi, yi in zip(X, Y):
        h_times = layer_forward(xi, W1_tr, 1)
        o_times = layer_forward(h_times, W2_tr, 2)
        pred_class = np.argmax(o_times)
        true_class = np.argmax(yi)
        if pred_class == true_class:
            correct += 1
    
    final_accuracy = correct / len(X)
    print(f"Final Accuracy: {final_accuracy:.3f} ({correct}/{len(X)})")

=== Training ===


WARNING    C:\Users\irtho\AppData\Local\Temp\ipykernel_21960\1577268996.py:157: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  L_non = 0.5 * λ * sum((o_times[j] - non_target_time)**2 for j in non_ids)
 [py.warnings]


Epoch 1/50 — avg loss=0.1794
             ‖W1‖=0.672, ‖W2‖=0.555
             Accuracy: 0.500



KeyboardInterrupt: 

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# Set random seed for reproducibility
np.random.seed(0)
torch.manual_seed(0)

# Dummy dataset (8 samples, 5 features)
X = np.array([
    [1, 0, 0, 1, 0],
    [0, 1, 0, 1, 0],
    [1, 1, 0, 1, 0],
    [0, 0, 1, 0, 1],
    [1, 0, 1, 0, 1],
    [0, 1, 1, 0, 1],
    [1, 1, 1, 1, 1],
    [0, 0, 0, 0, 0]
], dtype=np.float32)

y = np.array([0, 0, 0, 1, 1, 1, 2, 2])

# Hyperparameters
input_size = 5
hidden_size = 10
output_size = 3
learning_rate = 0.5
num_epochs = 200
margin = 0.5
layer_offset = 0.2

def spike_timing(w, input_time, layer_offset):
    return input_time + w + layer_offset

class SpikingNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.W1 = nn.Parameter(torch.randn(input_size, hidden_size) * 0.8)
        self.W2 = nn.Parameter(torch.randn(hidden_size + 1, output_size) * 0.8)

    def forward(self, x):
        # Layer 1: compute spike times
        x = x.unsqueeze(0) if len(x.shape) == 1 else x
        x_times = x.clone()
        h_times = spike_timing(self.W1.T, x_times, layer_offset)

        # Add bias input
        bias = torch.ones((x.shape[0], 1))
        h_with_bias = torch.cat([h_times, bias], dim=1)

        # Layer 2: output spike times
        o_times = spike_timing(self.W2.T, h_with_bias, layer_offset)
        return o_times

def ranking_loss(o_times, target_class, margin):
    correct_time = o_times[target_class]
    loss = 0.0
    for i, t in enumerate(o_times):
        if i != target_class:
            loss += torch.clamp(t - correct_time + margin, min=0)
    return loss

model = SpikingNet(input_size, hidden_size, output_size)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

print("Starting robust training...")

for epoch in range(1, num_epochs + 1):
    total_loss = 0
    correct = 0
    for i in range(len(X)):
        x = torch.tensor(X[i])
        target = y[i]

        o_times = model(x)
        loss = ranking_loss(o_times, target, margin)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        pred = torch.argmin(o_times).item()
        correct += (pred == target)

    if epoch % 20 == 1 or epoch == num_epochs:
        print(f"Epoch {epoch}/{num_epochs} — avg loss={total_loss / len(X):.4f}")
        print(f"             Accuracy: {correct / len(X):.3f}")
        print(f"             Sample output: {o_times.detach().numpy().round(3)}")
        print(f"             Target: {[o if i == target else '-' for i, o in enumerate(o_times.detach().numpy().round(3))]}")

# Final test
print("\nFinal Evaluation:")
correct = 0
for i in range(len(X)):
    x = torch.tensor(X[i])
    target = y[i]
    o_times = model(x)
    pred = torch.argmin(o_times).item()
    print(f"Sample {i}: Pred={pred}, True={target} {'✓' if pred == target else '✗'}")
    correct += (pred == target)

print(f"\nFinal Accuracy: {correct / len(X):.3f} ({correct}/{len(X)})")


=== Robust SNN Training with Enhanced Loss Function ===
Starting robust training...
Epoch 1/200 — avg loss=1.2006
             Accuracy: 0.000
             Sample output: [3.425, 3.561, 3.389]
             Target: [2.950, 2.000, 2.000]

Epoch 21/200 — avg loss=3.7211
             Accuracy: 0.000

Epoch 41/200 — avg loss=4.5120
             Accuracy: 0.000
             Sample output: [3.332, 4.237, 3.332]
             Target: [2.950, 2.000, 2.000]

Epoch 61/200 — avg loss=4.5120
             Accuracy: 0.000

Epoch 81/200 — avg loss=4.5120
             Accuracy: 0.000
             Sample output: [3.332, 4.237, 3.332]
             Target: [2.950, 2.000, 2.000]

Epoch 101/200 — avg loss=4.5120
             Accuracy: 0.000

Epoch 121/200 — avg loss=4.5120
             Accuracy: 0.000
             Sample output: [3.332, 4.237, 3.332]
             Target: [2.950, 2.000, 2.000]

Epoch 141/200 — avg loss=4.5120
             Accuracy: 0.000

Epoch 161/200 — avg loss=4.5120
             Accuracy:

In [ ]:
block1 = np.array([0.8, 0.7, 0.3, 0.6])
block2 = np.array([0.9, 0.5, 0.5, 0.2])
it = [block1 if i % 2 == 0 else block2 for i in range(8)]



X = [np.array([0.6, 0.2, 0.4, 0.8]) for _ in range(8)]

print(X)
print(it)

[array([0.6, 0.2, 0.4, 0.8]), array([0.6, 0.2, 0.4, 0.8]), array([0.6, 0.2, 0.4, 0.8]), array([0.6, 0.2, 0.4, 0.8]), array([0.6, 0.2, 0.4, 0.8]), array([0.6, 0.2, 0.4, 0.8]), array([0.6, 0.2, 0.4, 0.8]), array([0.6, 0.2, 0.4, 0.8])]
[array([0.8, 0.7, 0.3, 0.6]), array([0.9, 0.5, 0.5, 0.2]), array([0.8, 0.7, 0.3, 0.6]), array([0.9, 0.5, 0.5, 0.2]), array([0.8, 0.7, 0.3, 0.6]), array([0.9, 0.5, 0.5, 0.2]), array([0.8, 0.7, 0.3, 0.6]), array([0.9, 0.5, 0.5, 0.2])]


In [ ]:
import numpy as np
import warnings

# Suppress warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)
np.seterr(over='ignore', under='ignore')

# ----------------------------------------------------------------------------
# Spike timing functions (no Brian2 dependency)

def spike_timing(w, input_time, layer_offset):
    if input_time <= 0:
        return 5.0  # No spike if no input
    base_time = input_time
    timing_factor = np.tanh(w * 0.5)
    scaled_timing = base_time + timing_factor * 0.5
    return scaled_timing + layer_offset

def d_spike_timing_dw(w, input_time, layer_offset):
    if input_time <= 0:
        return 0.0
    tanh_val = np.tanh(w * 0.5)
    return 0.25 * (1 - tanh_val**2)
    
   

# ----------------------------------------------------------------------------
# Efficient forward pass without Brian2

def layer_forward_enhanced(inputs, W, layer_idx):
    aug_inputs = np.concatenate((inputs, [0.0]))
    n_in_plus_bias, n_out = W.shape
    output_times = np.zeros(n_out)

    for j in range(n_out):
        weighted_sum = 0.0
        total_weight = 0.0

        for i in range(n_in_plus_bias):
            input_time = aug_inputs[i] if i < len(inputs) else 0.1
            weight = W[i, j]
            contribution = spike_timing(weight, input_time, 0)
            weighted_sum += contribution * abs(weight)
            total_weight += abs(weight)

        if total_weight > 0:
            output_times[j] = (weighted_sum / total_weight) + layer_idx
        else:
            output_times[j] = 2.0 + layer_idx

    return output_times


# ----------------------------------------------------------------------------
# Training with efficient forward pass

def train_snn_robust(
    X, Y,
    W1_init, W2_init,
    epochs=100, lr=0.3,
    max_grad=5.0,
    w_min=-3.0, w_max=3.0,
    target_separation=0.5
):
    W1 = W1_init.copy()
    W2 = W2_init.copy()

    beta1, beta2 = 0.9, 0.999
    eps = 1e-8
    m1_W1 = np.zeros_like(W1)
    v1_W1 = np.zeros_like(W1)
    m1_W2 = np.zeros_like(W2)
    v1_W2 = np.zeros_like(W2)
    N = len(X)

    for ep in range(epochs):
        acc_dW1 = np.zeros_like(W1)
        acc_dW2 = np.zeros_like(W2)
        epoch_loss = 0.0

        for xi, yi in zip(X, Y):
            h_times = layer_forward_enhanced(xi, W1, 1)
            o_times = layer_forward_enhanced(h_times, W2, 2)

            target_idx = np.argmax(yi)
            target_time = o_times[target_idx]

            separation_losses = []
            for j in range(len(o_times)):
                if j != target_idx:
                    margin = o_times[j] - target_time
                    if margin < target_separation:
                        separation_losses.append((target_separation - margin) ** 2)
                    else:
                        separation_losses.append(0.0)

            target_loss = 0.5 * (target_time - yi[target_idx]) ** 2
            separation_loss = sum(separation_losses)
            L = target_loss + 2.0 * separation_loss
            epoch_loss += L

            delta_o = np.zeros_like(o_times)
            delta_o[target_idx] = (target_time - yi[target_idx])

            for j in range(len(o_times)):
                if j != target_idx:
                    margin = o_times[j] - target_time
                    if margin < target_separation:
                        delta_o[j] = 2.0 * (target_separation - margin)
                        delta_o[target_idx] -= 2.0 * (target_separation - margin)

            aug_h = np.concatenate((h_times, [0.1]))
            dW2 = np.zeros_like(W2)
            for k in range(W2.shape[0]):
                for j in range(W2.shape[1]):
                    input_time = aug_h[k]
                    dt_dw = d_spike_timing_dw(W2[k, j], input_time, 2)
                    dW2[k, j] = delta_o[j] * dt_dw * 10.0

            delta_h = np.zeros_like(h_times)
            for k in range(len(h_times)):
                for j in range(W2.shape[1]):
                    input_time = aug_h[k]
                    dt_dw = d_spike_timing_dw(W2[k, j], input_time, 2)
                    delta_h[k] += delta_o[j] * dt_dw * W2[k, j] * 0.1

            aug_xi = np.concatenate((xi, [0.1]))
            dW1 = np.zeros_like(W1)
            for i in range(W1.shape[0]):
                for k in range(W1.shape[1]):
                    input_time = aug_xi[i]
                    dt_dw = d_spike_timing_dw(W1[i, k], input_time, 1)
                    dW1[i, k] = delta_h[k] * dt_dw * 10.0

            acc_dW1 += dW1
            acc_dW2 += dW2

        acc_dW1 /= N
        acc_dW2 /= N
        acc_dW1 = np.clip(acc_dW1, -max_grad, max_grad)
        acc_dW2 = np.clip(acc_dW2, -max_grad, max_grad)

        m1_W1 = beta1 * m1_W1 + (1 - beta1) * acc_dW1
        m1_W2 = beta1 * m1_W2 + (1 - beta1) * acc_dW2
        v1_W1 = beta2 * v1_W1 + (1 - beta2) * (acc_dW1 ** 2)
        v1_W2 = beta2 * v1_W2 + (1 - beta2) * (acc_dW2 ** 2)

        m1_W1_corr = m1_W1 / (1 - beta1 ** (ep + 1))
        m1_W2_corr = m1_W2 / (1 - beta1 ** (ep + 1))
        v1_W1_corr = v1_W1 / (1 - beta2 ** (ep + 1))
        v1_W2_corr = v1_W2 / (1 - beta2 ** (ep + 1))

        W1 -= lr * m1_W1_corr / (np.sqrt(v1_W1_corr) + eps)
        W2 -= lr * m1_W2_corr / (np.sqrt(v1_W2_corr) + eps)

        W1 = np.clip(W1, w_min, w_max)
        W2 = np.clip(W2, w_min, w_max)

        if ep % 20 == 0 or ep == epochs - 1:
            print(f"Epoch {ep+1}/{epochs} — avg loss={epoch_loss/N:.4f}")
            correct = 0
            for xi, yi in zip(X, Y):
                h_times = layer_forward_enhanced(xi, W1, 1)
                o_times = layer_forward_enhanced(h_times, W2, 2)
                if np.argmax(o_times) == np.argmax(yi):
                    correct += 1
            print(f"             Accuracy: {correct / len(X):.3f}")
            if ep % 40 == 0:
                xi, yi = X[0], Y[0]
                h_times = layer_forward_enhanced(xi, W1, 1)
                o_times = layer_forward_enhanced(h_times, W2, 2)
                print(f"             Sample output: {o_times.round(3)}")
                print(f"             Target: {yi.round(3)}\n")

    return W1, W2

# ----------------------------------------------------------------------------
# Main execution


   




=== Robust SNN Training with Enhanced Loss Function ===
Starting robust training...
Epoch 1/200 — avg loss=0.9566
             Accuracy: 0.500
             Sample output: [3.329 3.298 3.35 ]
             Target: [2.95 2.   2.  ]

Epoch 21/200 — avg loss=4.2276
             Accuracy: 0.500
Epoch 41/200 — avg loss=5.2386
             Accuracy: 0.500
             Sample output: [4.234 3.329 4.234]
             Target: [2.95 2.   2.  ]

Epoch 61/200 — avg loss=5.2424
             Accuracy: 0.500
Epoch 81/200 — avg loss=5.2424
             Accuracy: 0.500
             Sample output: [4.237 3.332 4.237]
             Target: [2.95 2.   2.  ]

Epoch 101/200 — avg loss=5.2424
             Accuracy: 0.500
Epoch 121/200 — avg loss=5.2424
             Accuracy: 0.500
             Sample output: [4.237 3.332 4.237]
             Target: [2.95 2.   2.  ]

Epoch 141/200 — avg loss=5.2424
             Accuracy: 0.500
Epoch 161/200 — avg loss=5.2424
             Accuracy: 0.500
             Sample outpu

Trained W1: [[ 4.42270137e+00  2.72902627e+00  4.52251250e+00 -2.66418654e-02
  -1.32903905e-03  4.14133410e-02  4.61386303e+00  1.83720797e-01
  -7.51569272e-02  3.51829401e+00]
 [ 5.39742143e+00  4.22116997e+00  5.31105386e+00 -4.65205424e-02
  -1.70381678e-02  7.78778861e-02  5.08195082e+00  7.47894500e-03
  -9.03152356e-03  5.05393082e+00]
 [ 4.43345211e+00  2.17641016e+00  4.37444012e+00  7.41919002e-01
  -6.53028519e-02  9.41795776e-03  4.57591258e+00  1.17434939e-01
  -2.42172258e-01  2.89698446e+00]
 [ 2.96324751e+00  1.34519281e+00  3.20728606e+00 -1.25186459e+00
  -1.58785854e-03 -6.27490458e-02  4.34013077e+00 -6.69617214e-01
   9.15434146e-02  1.46241605e+00]]
Trained W2: [[-5.19009294e+00 -9.22196291e-01  1.06320763e-01]
 [-4.14733377e+00 -1.11731667e+00  7.87285482e-03]
 [-5.35983978e+00 -7.60399948e-01 -1.72060318e-03]
 [ 6.37550208e-01 -7.07602762e-01  1.22677933e-01]
 [-1.99769654e-02 -4.49491576e-01  9.30378392e-02]
 [ 3.01707533e-02 -1.04142396e+00  1.53061594e-01]
 [-6.27254161e+00 -5.01911927e-01  1.12362536e-02]
 [ 6.85429277e-01 -1.12952211e+00  6.65988918e-02]
 [ 1.42345211e-01 -4.74105845e-01  7.04198093e-02]
 [-4.10253561e+00 -5.98382591e-01  1.40683024e-01]]

In [16]:
# assuming you have W1_tr, W2_tr from training

def relu(x):
    return np.maximum(0, x)

# convert X_test from scaler, y_test from integer classes
# use X_test from the same split
# we still have y[indices] from before:
_, X_test_raw, _, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

# still assuming you have W1_tr, W2_tr, relu, X_test, y_test

# initialize counts
class_totals = {0: 0, 1: 0, 2: 0}
class_correct = {0: 0, 1: 0, 2: 0}

total = len(X_test)

for i in range(total):
    x = X_test[i]
    h = relu(np.dot(x, W1_tr))         # hidden layer
    out = np.dot(h, W2_tr)             # output layer

    pred_class = np.argmax(out)
    true_class = y_test[i]
    
    class_totals[true_class] += 1
    if pred_class == true_class:
        class_correct[true_class] += 1

# print per-class accuracy
for c in [0, 1, 2]:
    total_c = class_totals[c]
    correct_c = class_correct[c]
    acc_c = correct_c / total_c if total_c > 0 else 0
    print(f"Class {c}: {correct_c}/{total_c} correct ({acc_c:.2%})")

# also global accuracy
overall_correct = sum(class_correct.values())
# accuracy = overall_correct / total
# print(f"Overall accuracy: {accuracy:.2%}")



NameError: name 'W1_tr' is not defined

In [ ]:
from brian2 import *
import numpy as np
import logging
import warnings
from sklearn import datasets
from sklearn.utils import shuffle

# suppress overflow warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)
numpy.seterr(over='ignore', under='ignore')
logging.getLogger('brian2').setLevel(logging.ERROR)

# ----------------------------------------------------------------------------
# Spike timing and derivative

start_scope()
defaultclock.dt = 0.001*ms

@implementation('numpy', discard_units=True)
@check_units(w=1, global_clock=1, layer=1, sum=1, spikes_received=1, result=1)
def spike_timing(w, global_clock, layer, sum, spikes_received):
    x = global_clock % 1
    if w >= 0:
        return np.power(x, (1 - w), where=(x>0), out=np.zeros_like(x))
    else:
        return 1 - np.power((1 - x), (1 + w), where=(x<1), out=np.ones_like(x))

@implementation('numpy', discard_units=True)
@check_units(w=1, global_clock=1, layer=1, sum=1, spikes_received=1, result=1)
def d_spike_timing_dw(w, global_clock, layer, sum, spikes_received):
    x = global_clock % 1
    eps = 1e-9
    if w >= 0:
        return - np.power(x, (1 - w), where=(x>0), out=np.zeros_like(x)) * np.log(x + eps)
    else:
        return - np.power((1 - x), (1 + w), where=(x<1), out=np.ones_like(x)) * np.log(1 - x + eps)

# ----------------------------------------------------------------------------
# Layer forward pass

def layer_forward(inputs, W, layer_idx):
    n_in, n_out = W.shape
    out_times = []
    for j in range(n_out):
        start_scope()
        defaultclock.dt = 0.0001*ms
        G = NeuronGroup(1, '''
            v : 1
            sum : 1
            sr : 1
            scheduled_time : second
            global_clock : 1
        ''', threshold='v>1', reset='v=0', method='exact')
        G.v = G.sum = G.sr = 0
        G.global_clock = 0
        G.scheduled_time = 1e9*second
        stim = SpikeGeneratorGroup(n_in, indices=list(range(n_in)), times=inputs*ms)
        S = Synapses(stim, G, '''w:1
            layer:1''', on_pre='''
            sr += 1
            sum += spike_timing(w, global_clock, layer, sum, sr)
            scheduled_time = (sum/sr + layer)*ms
        ''')
        S.connect(True)
        S.w = W[:, j]
        S.layer = layer_idx
        G.run_regularly('''
            v = int(abs(t - scheduled_time) < 0.0005*ms) * 1.2
            global_clock += 0.001
        ''', dt=0.001*ms)
        mon = SpikeMonitor(G)
        run(5*ms)
        ts = mon.spike_trains()[0]
        t0 = float(ts[0]/ms) if len(ts)>0 else float(5.0)
        out_times.append(t0)
    return np.array(out_times)

# ----------------------------------------------------------------------------
# Training loop

def train_snn_backprop(
    X, Y,
    W1_init, W2_init,
    epochs=10, lr=0.1, max_grad=20.0, w_min=-20.0, w_max=20.0
):
    W1 = W1_init.copy()
    W2 = W2_init.copy()
    layer1_idx = 1
    layer2_idx = 2

    for ep in range(epochs):
        print(f"Epoch {ep+1}/{epochs}")
        for xi, yi in zip(X, Y):
            h_times = layer_forward(xi, W1, layer1_idx)
            o_times = layer_forward(h_times, W2, layer2_idx)
            L = 0.5 * np.sum((o_times - yi)**2)
            dW2 = np.zeros_like(W2)
            delta_o = (o_times - yi)
            for k in range(W2.shape[0]):
                for j in range(W2.shape[1]):
                    dW2[k,j] = delta_o[j] * d_spike_timing_dw(
                        W2[k,j], h_times[k], layer2_idx, 0, 1)
            delta_h = np.zeros_like(h_times)
            for k in range(len(h_times)):
                for j in range(W2.shape[1]):
                    dt_dw = d_spike_timing_dw(W2[k,j], h_times[k], layer2_idx, 0, 1)
                    delta_h[k] += delta_o[j] * W2[k,j] * dt_dw
            dW1 = np.zeros_like(W1)
            for i in range(W1.shape[0]):
                for k in range(W1.shape[1]):
                    dW1[i,k] = delta_h[k] * d_spike_timing_dw(
                        W1[i,k], xi[i], layer1_idx, 0, 1)
            dW1 = np.clip(dW1, -max_grad, max_grad)
            dW2 = np.clip(dW2, -max_grad, max_grad)
            W1 = np.clip(W1 - lr * dW1, w_min, w_max)
            W2 = np.clip(W2 - lr * dW2, w_min, w_max)
        print(f" End Epoch {ep+1}: W1 norm={np.linalg.norm(W1):.3f}, W2 norm={np.linalg.norm(W2):.3f}\n")
    return W1, W2

# ----------------------------------------------------------------------------
# Data preparation for Iris

def scale_inputs(X, low=0.05, high=0.095):
    # min-max scale each feature to [low, high]
    min_vals = X.min(axis=0)
    max_vals = X.max(axis=0)
    return low + (X - min_vals) * (high - low) / (max_vals - min_vals)


def encode_targets(y, class_count=3, spike_low=2.05, spike_high=2.95):
    # one-hot-like mapping: for each sample, array of length class_count
    Y = []
    for label in y:
        arr = np.full(class_count, spike_low)
        arr[label] = spike_high
        Y.append(arr)
    return np.array(Y)

if __name__ == "__main__":
    # load and shuffle Iris
    iris = datasets.load_iris()
    X_raw, y_raw = shuffle(iris.data, iris.target, random_state=42)

    # scale inputs and encode targets
    X_scaled = scale_inputs(X_raw)
    Y_encoded = encode_targets(y_raw)

    # split 80/20
    split = int(0.8 * len(X_scaled))
    X_train, X_test = X_scaled[:split], X_scaled[split:]
    Y_train, Y_test = Y_encoded[:split], Y_encoded[split:]

    # initialize weights
    W1_0 = np.random.randn(X_scaled.shape[1], 10) * 0.1
    W2_0 = np.random.randn(10, Y_encoded.shape[1]) * 0.1

    # train
    W1_tr, W2_tr = train_snn_backprop(
        X_train, Y_train, W1_0, W2_0,
        epochs=20, lr=0.4
    )

    # evaluate on test set
    print("Evaluating on test set...")
    for xi, yi_true in zip(X_test, Y_test):
        o_times = layer_forward(xi, W2_tr, layer_idx=2)
        print(f"Input: {xi}, Pred times: {o_times}, True times: {yi_true}")


Epoch 1/20


KeyboardInterrupt: 


        ---     

Old scratch below urd 2x2 model and such

            ---         

In [4]:
from brian2 import *
import numpy as np
import logging
import warnings

# suppress overflow warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)
numpy.seterr(over='ignore', under='ignore')
logging.getLogger('brian2').setLevel(logging.ERROR)

# ----------------------------------------------------------------------------
# Spike timing and derivative

start_scope()
defaultclock.dt = 0.0001*ms

@implementation('numpy', discard_units=True)
@check_units(w=1, global_clock=1, layer=1, sum=1, spikes_received=1, result=1)
def spike_timing(w, global_clock, layer, sum, spikes_received):
    x = global_clock % 1
    if w >= 0:
        return np.power(x, (1 - w), where=(x>0), out=np.zeros_like(x))
    else:
        return 1 - np.power((1 - x), (1 + w), where=(x<1), out=np.ones_like(x))

@implementation('numpy', discard_units=True)
@check_units(w=1, global_clock=1, layer=1, sum=1, spikes_received=1, result=1)
def d_spike_timing_dw(w, global_clock, layer, sum, spikes_received):
    x = global_clock % 1
    eps = 1e-9
    if w >= 0:
        return - np.power(x, (1 - w), where=(x>0), out=np.zeros_like(x)) * np.log(x + eps)
    else:
        return - np.power((1 - x), (1 + w), where=(x<1), out=np.ones_like(x)) * np.log(1 - x + eps)

# ----------------------------------------------------------------------------
# Batched forward pass: process multiple samples simultaneously

def layer_forward_batched(inputs_batch, W, layer_idx, batch_size):
    """
    inputs_batch: list of arrays, each of shape (n_in,) - spike times from previous layer
    W: weight matrix shape (n_in, n_out)
    layer_idx: integer layer number
    batch_size: number of samples to process
    returns: list of arrays, each of shape (n_out,) - output spike times
    """
    n_in, n_out = W.shape
    results = []
    
    # Process each sample individually but collect results for batching
    for batch_idx, inputs in enumerate(inputs_batch):
        sample_results = []
        
        for j in range(n_out):
            start_scope()
            defaultclock.dt = 0.0001*ms
            
            # Create single neuron for this output
            G = NeuronGroup(1, '''
                v : 1
                sum : 1
                sr : 1
                scheduled_time : second
                global_clock : 1
            ''', threshold='v>1', reset='v=0', method='exact')
            
            G.v = G.sum = G.sr = 0
            G.global_clock = 0
            G.scheduled_time = 1e9*second
            
            # Create spike inputs for this sample
            stim = SpikeGeneratorGroup(n_in, indices=list(range(n_in)), times=inputs*ms)
            
            # Create synapses
            S = Synapses(stim, G, '''
                w : 1
                layer : 1
            ''', on_pre='''
                sr += 1
                sum += spike_timing(w, global_clock, layer, sum, sr)
                scheduled_time = (sum/sr + layer)*ms
            ''')
            
            S.connect(True)
            S.w = W[:, j]
            S.layer = layer_idx
            
            # Update global clock and voltage
            G.run_regularly('''
                v = int(abs(t - scheduled_time) < 0.0005*ms) * 1.2
                global_clock += 0.001
            ''', dt=0.001*ms)
            
            # Monitor spikes
            mon = SpikeMonitor(G)
            
            # Run simulation
            run(5*ms)
            
            # Extract spike time
            ts = mon.spike_trains()[0]
            t0 = float(ts[0]/ms) if len(ts) > 0 else float(5.0)
            sample_results.append(t0)
        
        results.append(np.array(sample_results))
    
    return results

# ----------------------------------------------------------------------------
# Batched training loop

def train_snn_backprop_batched(
    X, Y,        # lists of input arrays (4,) and target (3,)
    W1_init, W2_init,
    epochs=10, lr=0.1, max_grad=20.0, w_min=-20.0, w_max=20.0,
    batch_size=4
):
    W1 = W1_init.copy()
    W2 = W2_init.copy()
    layer1_idx = 1
    layer2_idx = 2
    
    n_samples = len(X)
    
    for ep in range(epochs):
        print(f"Epoch {ep+1}/{epochs}")
        
        # Process in batches
        for batch_start in range(0, n_samples, batch_size):
            batch_end = min(batch_start + batch_size, n_samples)
            current_batch_size = batch_end - batch_start
            
            # Get batch data
            X_batch = X[batch_start:batch_end]
            Y_batch = Y[batch_start:batch_end]
            
            # Forward pass through both layers
            h_times_batch = layer_forward_batched(X_batch, W1, layer1_idx, current_batch_size)
            o_times_batch = layer_forward_batched(h_times_batch, W2, layer2_idx, current_batch_size)
            
            # Accumulate gradients for this batch
            dW1_batch = np.zeros_like(W1)
            dW2_batch = np.zeros_like(W2)
            total_loss = 0.0
            
            for i, (xi, yi, h_times, o_times) in enumerate(zip(X_batch, Y_batch, h_times_batch, o_times_batch)):
                # Loss for this sample
                L = 0.5 * np.sum((o_times - yi)**2)
                total_loss += L
                
                # Gradients for W2
                dW2_sample = np.zeros_like(W2)
                delta_o = (o_times - yi)
                
                for k in range(W2.shape[0]):
                    for j in range(W2.shape[1]):
                        dW2_sample[k,j] = delta_o[j] * d_spike_timing_dw(
                            W2[k,j], h_times[k], layer2_idx, 0, 1)
                
                # Hidden deltas
                delta_h = np.zeros_like(h_times)
                for k in range(len(h_times)):
                    for j in range(W2.shape[1]):
                        dt_dw = d_spike_timing_dw(W2[k,j], h_times[k], layer2_idx, 0, 1)
                        delta_h[k] += delta_o[j] * W2[k,j] * dt_dw
                
                # Gradients for W1
                dW1_sample = np.zeros_like(W1)
                for ii in range(W1.shape[0]):
                    for k in range(W1.shape[1]):
                        dW1_sample[ii,k] = delta_h[k] * d_spike_timing_dw(
                            W1[ii,k], xi[ii], layer1_idx, 0, 1)
                
                # Accumulate gradients
                dW1_batch += dW1_sample
                dW2_batch += dW2_sample
                
                # Print individual sample results
                print(f"  Sample {batch_start+i}: Input: {xi}, Pred: {o_times}, Target: {yi}, Loss: {L:.4f}")
            
            # Average gradients over batch
            dW1_batch /= current_batch_size
            dW2_batch /= current_batch_size
            
            # Clip and update weights
            dW1_batch = np.clip(dW1_batch, -max_grad, max_grad)
            dW2_batch = np.clip(dW2_batch, -max_grad, max_grad)
            W1 = np.clip(W1 - lr * dW1_batch, w_min, w_max)
            W2 = np.clip(W2 - lr * dW2_batch, w_min, w_max)
            
            avg_loss = total_loss / current_batch_size
            print(f"  Batch {batch_start//batch_size + 1}: Avg Loss: {avg_loss:.4f}")
        
        print(f" End Epoch {ep+1}: W1 norm={np.linalg.norm(W1):.3f}, W2 norm={np.linalg.norm(W2):.3f}\n")
    
    return W1, W2

if __name__ == "__main__":
    # example usage with fixed input/target pairs
    # 4 inputs per sample, constant across 8 samples
    X = [np.array([0.6, 0.2, 0.4, 0.8]) for _ in range(8)]
    # 3-targets (network outputs 3 values): use desired spike times [2.1, 2.0, 1.0]
    Y = [np.array([2.1, 2.9, 2.1]) for _ in range(8)]
    # initialize weights
    W1_0 = np.random.randn(4, 10) * 0.1
    W2_0 = np.random.randn(10, 3) * 0.1
    
    print("Initial weights - W1 norm:", np.linalg.norm(W1_0))
    print("Initial weights - W2 norm:", np.linalg.norm(W2_0))
    print("Sample W1 values:", W1_0[:2, :3])
    print("Sample W2 values:", W2_0[:3, :])
    
    # train with batching
    W1_tr, W2_tr = train_snn_backprop_batched(X, Y, W1_0, W2_0,
                                             epochs=5, lr=0.4, batch_size=4)
    print("Trained W1:", W1_tr)
    print("Trained W2:", W2_tr)

Initial weights - W1 norm: 0.7618281992028296
Initial weights - W2 norm: 0.5620876397440523
Sample W1 values: [[-0.12467743  0.01261521  0.00491398]
 [ 0.0071659   0.07718885  0.11401133]]
Sample W2 values: [[-0.08962412  0.05624603  0.26515587]
 [-0.12227563  0.00762061  0.04028084]
 [ 0.07210636 -0.01546034  0.13874489]]
Epoch 1/5
  Sample 0: Input: [0.6 0.2 0.4 0.8], Pred: [2.494 2.501 2.514], Target: [2.1 2.9 2.1], Loss: 0.2429
  Sample 1: Input: [0.6 0.2 0.4 0.8], Pred: [2.494 2.501 2.514], Target: [2.1 2.9 2.1], Loss: 0.2429
  Sample 2: Input: [0.6 0.2 0.4 0.8], Pred: [2.494 2.501 2.514], Target: [2.1 2.9 2.1], Loss: 0.2429
  Sample 3: Input: [0.6 0.2 0.4 0.8], Pred: [2.494 2.501 2.514], Target: [2.1 2.9 2.1], Loss: 0.2429
  Batch 1: Avg Loss: 0.2429
  Sample 4: Input: [0.6 0.2 0.4 0.8], Pred: [2.472 2.522 2.491], Target: [2.1 2.9 2.1], Loss: 0.2171
  Sample 5: Input: [0.6 0.2 0.4 0.8], Pred: [2.472 2.522 2.491], Target: [2.1 2.9 2.1], Loss: 0.2171
  Sample 6: Input: [0.6 0.2 0.4

In [5]:
from brian2 import *
import numpy as np
import logging
import warnings
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# suppress overflow warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)
numpy.seterr(over='ignore', under='ignore')
logging.getLogger('brian2').setLevel(logging.ERROR)

# ----------------------------------------------------------------------------
# Iris dataset preprocessing functions

def load_and_prepare_iris():
    """
    Load Iris dataset and prepare it for SNN training.
    
    Returns:
        X_train, X_test, y_train, y_test: Training and test splits
        All inputs scaled to [0.05, 0.95] range
        All outputs converted to spike timing format
    """
    # Load the iris dataset
    iris = load_iris()
    X, y = iris.data, iris.target
    
    print(f"Original dataset shape: {X.shape}")
    print(f"Feature names: {iris.feature_names}")
    print(f"Target names: {iris.target_names}")
    print(f"Target distribution: {np.bincount(y)}")
    
    # Standardize inputs to [0.05, 0.95] range
    scaler = MinMaxScaler(feature_range=(0.05, 0.95))
    X_scaled = scaler.fit_transform(X)
    
    # Convert class labels to spike timing format
    y_spike_times = convert_labels_to_spike_times(y)
    
    # Create train/test split with shuffling
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y_spike_times, 
        test_size=0.2, 
        random_state=42, 
        shuffle=True,
        stratify=y  # Ensure balanced split across classes
    )
    
    print(f"Training set size: {len(X_train)}")
    print(f"Test set size: {len(X_test)}")
    print(f"Input range: [{X_scaled.min():.3f}, {X_scaled.max():.3f}]")
    
    return X_train, X_test, y_train, y_test, scaler

def convert_labels_to_spike_times(labels):
    """
    Convert class labels (0, 1, 2) to spike timing format.
    
    Class 0 (setosa): [2.0, 3.0, 2.0]
    Class 1 (versicolor): [2.0, 2.0, 3.0] 
    Class 2 (virginica): [3.0, 2.0, 2.0]
    
    Args:
        labels: array of class labels (0, 1, 2)
        
    Returns:
        array of spike time arrays, shape (n_samples, 3)
    """
    # Define spike time patterns for each class
    spike_patterns = {
        0: np.array([2.0, 3.0, 2.0]),  # setosa
        1: np.array([2.0, 2.0, 3.0]),  # versicolor  
        2: np.array([3.0, 2.0, 2.0])   # virginica
    }
    
    result = []
    for label in labels:
        result.append(spike_patterns[label].copy())
    
    return np.array(result)

def evaluate_model(X_test, y_test, W1, W2):
    """
    Evaluate the trained model on test data.
    
    Args:
        X_test: Test input data
        y_test: True test labels (spike timing format)
        W1, W2: Trained weights
        
    Returns:
        accuracy: Classification accuracy
        predictions: Predicted class labels
        true_labels: True class labels
    """
    # Convert test data to lists for the forward pass
    X_test_list = [x for x in X_test]
    
    # Forward pass through the network
    h_times_batch = layer_forward_batched(X_test_list, W1, 1, len(X_test_list))
    o_times_batch = layer_forward_batched(h_times_batch, W2, 2, len(h_times_batch))
    
    # Convert spike times back to class predictions
    predictions = []
    true_labels = []
    
    for i, (pred_times, true_times) in enumerate(zip(o_times_batch, y_test)):
        # Find the output with the earliest spike time (most active)
        pred_class = np.argmin(pred_times)
        true_class = np.argmin(true_times)
        
        predictions.append(pred_class)
        true_labels.append(true_class)
    
    # Calculate accuracy
    accuracy = np.mean(np.array(predictions) == np.array(true_labels))
    
    return accuracy, predictions, true_labels

def print_classification_report(true_labels, predictions):
    """
    Print detailed classification results.
    """
    class_names = ['setosa', 'versicolor', 'virginica']
    
    print("\nClassification Report:")
    print("=" * 50)
    
    for i, class_name in enumerate(class_names):
        true_positive = sum(1 for t, p in zip(true_labels, predictions) if t == i and p == i)
        false_positive = sum(1 for t, p in zip(true_labels, predictions) if t != i and p == i)
        false_negative = sum(1 for t, p in zip(true_labels, predictions) if t == i and p != i)
        
        precision = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0
        recall = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
        
        print(f"{class_name:12} - Precision: {precision:.3f}, Recall: {recall:.3f}, F1: {f1:.3f}")
    
    # Overall accuracy
    accuracy = sum(1 for t, p in zip(true_labels, predictions) if t == p) / len(true_labels)
    print(f"\nOverall Accuracy: {accuracy:.3f}")

# ----------------------------------------------------------------------------
# Spike timing and derivative

start_scope()
defaultclock.dt = 0.0001*ms

@implementation('numpy', discard_units=True)
@check_units(w=1, global_clock=1, layer=1, sum=1, spikes_received=1, result=1)
def spike_timing(w, global_clock, layer, sum, spikes_received):
    x = global_clock % 1
    if w >= 0:
        return np.power(x, (1 - w), where=(x>0), out=np.zeros_like(x))
    else:
        return 1 - np.power((1 - x), (1 + w), where=(x<1), out=np.ones_like(x))

@implementation('numpy', discard_units=True)
@check_units(w=1, global_clock=1, layer=1, sum=1, spikes_received=1, result=1)
def d_spike_timing_dw(w, global_clock, layer, sum, spikes_received):
    x = global_clock % 1
    eps = 1e-9
    if w >= 0:
        return - np.power(x, (1 - w), where=(x>0), out=np.zeros_like(x)) * np.log(x + eps)
    else:
        return - np.power((1 - x), (1 + w), where=(x<1), out=np.ones_like(x)) * np.log(1 - x + eps)

# ----------------------------------------------------------------------------
# Batched forward pass: process multiple samples simultaneously

def layer_forward_batched(inputs_batch, W, layer_idx, batch_size):
    """
    inputs_batch: list of arrays, each of shape (n_in,) - spike times from previous layer
    W: weight matrix shape (n_in, n_out)
    layer_idx: integer layer number
    batch_size: number of samples to process
    returns: list of arrays, each of shape (n_out,) - output spike times
    """
    n_in, n_out = W.shape
    results = []
    
    # Process each sample individually but collect results for batching
    for batch_idx, inputs in enumerate(inputs_batch):
        sample_results = []
        
        for j in range(n_out):
            start_scope()
            defaultclock.dt = 0.0001*ms
            
            # Create single neuron for this output
            G = NeuronGroup(1, '''
                v : 1
                sum : 1
                sr : 1
                scheduled_time : second
                global_clock : 1
            ''', threshold='v>1', reset='v=0', method='exact')
            
            G.v = G.sum = G.sr = 0
            G.global_clock = 0
            G.scheduled_time = 1e9*second
            
            # Create spike inputs for this sample
            stim = SpikeGeneratorGroup(n_in, indices=list(range(n_in)), times=inputs*ms)
            
            # Create synapses
            S = Synapses(stim, G, '''
                w : 1
                layer : 1
            ''', on_pre='''
                sr += 1
                sum += spike_timing(w, global_clock, layer, sum, sr)
                scheduled_time = (sum/sr + layer)*ms
            ''')
            
            S.connect(True)
            S.w = W[:, j]
            S.layer = layer_idx
            
            # Update global clock and voltage
            G.run_regularly('''
                v = int(abs(t - scheduled_time) < 0.0005*ms) * 1.2
                global_clock += 0.001
            ''', dt=0.001*ms)
            
            # Monitor spikes
            mon = SpikeMonitor(G)
            
            # Run simulation
            run(5*ms)
            
            # Extract spike time
            ts = mon.spike_trains()[0]
            t0 = float(ts[0]/ms) if len(ts) > 0 else float(5.0)
            sample_results.append(t0)
        
        results.append(np.array(sample_results))
    
    return results

# ----------------------------------------------------------------------------
# Batched training loop

def train_snn_backprop_batched(
    X, Y,        # lists of input arrays (4,) and target (3,)
    W1_init, W2_init,
    epochs=10, lr=0.1, max_grad=20.0, w_min=-20.0, w_max=20.0,
    batch_size=4
):
    W1 = W1_init.copy()
    W2 = W2_init.copy()
    layer1_idx = 1
    layer2_idx = 2
    
    n_samples = len(X)
    
    for ep in range(epochs):
        print(f"Epoch {ep+1}/{epochs}")
        
        # Process in batches
        for batch_start in range(0, n_samples, batch_size):
            batch_end = min(batch_start + batch_size, n_samples)
            current_batch_size = batch_end - batch_start
            
            # Get batch data
            X_batch = X[batch_start:batch_end]
            Y_batch = Y[batch_start:batch_end]
            
            # Forward pass through both layers
            h_times_batch = layer_forward_batched(X_batch, W1, layer1_idx, current_batch_size)
            o_times_batch = layer_forward_batched(h_times_batch, W2, layer2_idx, current_batch_size)
            
            # Accumulate gradients for this batch
            dW1_batch = np.zeros_like(W1)
            dW2_batch = np.zeros_like(W2)
            total_loss = 0.0
            
            for i, (xi, yi, h_times, o_times) in enumerate(zip(X_batch, Y_batch, h_times_batch, o_times_batch)):
                # Loss for this sample
                L = 0.5 * np.sum((o_times - yi)**2)
                total_loss += L
                
                # Gradients for W2
                dW2_sample = np.zeros_like(W2)
                delta_o = (o_times - yi)
                
                for k in range(W2.shape[0]):
                    for j in range(W2.shape[1]):
                        dW2_sample[k,j] = delta_o[j] * d_spike_timing_dw(
                            W2[k,j], h_times[k], layer2_idx, 0, 1)
                
                # Hidden deltas
                delta_h = np.zeros_like(h_times)
                for k in range(len(h_times)):
                    for j in range(W2.shape[1]):
                        dt_dw = d_spike_timing_dw(W2[k,j], h_times[k], layer2_idx, 0, 1)
                        delta_h[k] += delta_o[j] * W2[k,j] * dt_dw
                
                # Gradients for W1
                dW1_sample = np.zeros_like(W1)
                for ii in range(W1.shape[0]):
                    for k in range(W1.shape[1]):
                        dW1_sample[ii,k] = delta_h[k] * d_spike_timing_dw(
                            W1[ii,k], xi[ii], layer1_idx, 0, 1)
                
                # Accumulate gradients
                dW1_batch += dW1_sample
                dW2_batch += dW2_sample
                
                # Print individual sample results
                print(f"  Sample {batch_start+i}: Input: {xi}, Pred: {o_times}, Target: {yi}, Loss: {L:.4f}")
            
            # Average gradients over batch
            dW1_batch /= current_batch_size
            dW2_batch /= current_batch_size
            
            # Clip and update weights
            dW1_batch = np.clip(dW1_batch, -max_grad, max_grad)
            dW2_batch = np.clip(dW2_batch, -max_grad, max_grad)
            W1 = np.clip(W1 - lr * dW1_batch, w_min, w_max)
            W2 = np.clip(W2 - lr * dW2_batch, w_min, w_max)
            
            avg_loss = total_loss / current_batch_size
            print(f"  Batch {batch_start//batch_size + 1}: Avg Loss: {avg_loss:.4f}")
        
        print(f" End Epoch {ep+1}: W1 norm={np.linalg.norm(W1):.3f}, W2 norm={np.linalg.norm(W2):.3f}\n")
    
    return W1, W2

if __name__ == "__main__":
    # Load and prepare Iris dataset
    print("Loading and preparing Iris dataset...")
    X_train, X_test, y_train, y_test, scaler = load_and_prepare_iris()
    
    # Convert to lists for training function
    X_train_list = [x for x in X_train]
    y_train_list = [y for y in y_train]
    
    # Initialize weights for 4 inputs -> 10 hidden -> 3 outputs
    W1_0 = np.random.randn(4, 10) * 0.1
    W2_0 = np.random.randn(10, 3) * 0.1
    
    print(f"\nInitial weights - W1 shape: {W1_0.shape}, W2 shape: {W2_0.shape}")
    print(f"Training samples: {len(X_train_list)}")
    
    # Train the network
    print("\nStarting training...")
    W1_tr, W2_tr = train_snn_backprop_batched(
        X_train_list, y_train_list, W1_0, W2_0,
        epochs=10, lr=0.3, batch_size=8
    )
    
    # Evaluate on test set
    print("\nEvaluating on test set...")
    accuracy, predictions, true_labels = evaluate_model(X_test, y_test, W1_tr, W2_tr)
    
    print(f"\nTest Accuracy: {accuracy:.3f}")
    print_classification_report(true_labels, predictions)
    
    # Show some example predictions
    print("\nSample predictions:")
    print("=" * 60)
    class_names = ['setosa', 'versicolor', 'virginica']
    
    for i in range(min(10, len(X_test))):
        pred_class = predictions[i]
        true_class = true_labels[i]
        correct = "✓" if pred_class == true_class else "✗"
        
        print(f"Sample {i+1}: True={class_names[true_class]}, Pred={class_names[pred_class]} {correct}")
    
    print(f"\nFinal trained weights:")
    print(f"W1 norm: {np.linalg.norm(W1_tr):.3f}")
    print(f"W2 norm: {np.linalg.norm(W2_tr):.3f}")

Loading and preparing Iris dataset...
Original dataset shape: (150, 4)
Feature names: ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
Target names: ['setosa' 'versicolor' 'virginica']
Target distribution: [50 50 50]
Training set size: 120
Test set size: 30
Input range: [0.050, 0.950]

Initial weights - W1 shape: (4, 10), W2 shape: (10, 3)
Training samples: 120

Starting training...
Epoch 1/10
  Sample 0: Input: [0.075      0.3875     0.11101695 0.0875    ], Pred: [2.202 2.184 2.188], Target: [2. 3. 2.], Loss: 0.3710
  Sample 1: Input: [0.2        0.2375     0.58389831 0.65      ], Pred: [2.452 2.429 2.435], Target: [3. 2. 2.], Loss: 0.3368
  Sample 2: Input: [0.675      0.35       0.62966102 0.5375    ], Pred: [2.576 2.553 2.559], Target: [2. 2. 3.], Loss: 0.4160
  Sample 3: Input: [0.2        0.4625     0.12627119 0.05      ], Pred: [2.247 2.228 2.233], Target: [2. 3. 2.], Loss: 0.3556
  Sample 4: Input: [0.35       0.2375     0.50762712 0.5       ],

WARNING    C:\Users\irtho\AppData\Local\Temp\ipykernel_16140\3988774636.py:132: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  true_positive = sum(1 for t, p in zip(true_labels, predictions) if t == i and p == i)
 [py.warnings]
WARNING    C:\Users\irtho\AppData\Local\Temp\ipykernel_16140\3988774636.py:133: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  false_positive = sum(1 for t, p in zip(true_labels, predictions) if t != i and p == i)
 [py.warnings]
WARNING    C:\Users\irtho\AppData\Local\Temp\ipykernel_16140\3988774636.py:134: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  false_negative


Test Accuracy: 0.333

Classification Report:
setosa       - Precision: 0.000, Recall: 0.000, F1: 0.000
versicolor   - Precision: 0.345, Recall: 1.000, F1: 0.513
virginica    - Precision: 0.000, Recall: 0.000, F1: 0.000

Overall Accuracy: 0.333

Sample predictions:
Sample 1: True=setosa, Pred=versicolor ✗
Sample 2: True=versicolor, Pred=versicolor ✓
Sample 3: True=setosa, Pred=versicolor ✗
Sample 4: True=setosa, Pred=versicolor ✗
Sample 5: True=setosa, Pred=versicolor ✗
Sample 6: True=setosa, Pred=virginica ✗
Sample 7: True=setosa, Pred=versicolor ✗
Sample 8: True=setosa, Pred=versicolor ✗
Sample 9: True=versicolor, Pred=versicolor ✓
Sample 10: True=setosa, Pred=versicolor ✗

Final trained weights:
W1 norm: 97.445
W2 norm: 33.373


^Current workings

In [1]:
from brian2 import *
import numpy as np
import logging

logging.getLogger('brian2').setLevel(logging.ERROR)

start_scope()
defaultclock.dt = 0.0001*ms

In [2]:
# -----------------------------------------------------------------------------spike_timing + its derivative
# Functions used in brian2
@implementation('numpy', discard_units=True)
@check_units(w=1, global_clock=1, layer=1, sum=1, spikes_received=1, result=1)
def spike_timing(w, global_clock, layer, sum, spikes_received):
    x = global_clock % 1
    if w >= 0:
        return x**(1 - w)
    else:
        return 1 - (1 - x)**(1 + w)

@implementation('numpy', discard_units=True)
@check_units(w=1, global_clock=1, layer=1, sum=1, spikes_received=1, result=1)
def d_spike_timing_dw(w, global_clock, layer, sum, spikes_received):
    x = global_clock % 1
    eps = 1e-9
    if w >= 0:
        return - x**(1 - w) * np.log(x + eps)
    else:
        return - (1 - x)**(1 + w) * np.log(1 - x + eps)

def dsigmoid(z):
    s = 1/(1 + np.exp(-z))
    return s*(1 - s)

In [3]:
# -----------------------------------------------------------------------------
# 2) mini_urd forward: returns hidden‐spike‐time only
# -----------------------------------------------------------------------------
def mini_urd(inputs, w):
    n_input  = 2
    n_hidden = 2
    n_total  = n_input + n_hidden

    G = NeuronGroup(
        n_total,
        '''
        v               : 1
        sum             : 1
        sr              : 1
        scheduled_time  : second
        global_clock    : 1
        ''',
        threshold='v>1', reset='v=0', method='exact'
    )
    G.v = 0; G.sum = 0; G.sr = 0
    G.global_clock = 0
    G.scheduled_time = 1e9*second

    stim = SpikeGeneratorGroup(n_input, indices=[i for i in range(n_input)], times=inputs*ms)

    # first layer has fixed identity weights
    S1 = Synapses(stim, G[:n_input],
        'layer:1', on_pre='''
        sr += 1
        sum += spike_timing(1, global_clock, layer, sum, sr)
        scheduled_time = (1/(1+exp(-(sum/sr))) + layer)*ms
        '''
    )
    S1.connect(j='i')
    S1.layer = 0

    # trainable synapse 2→hidden
    S2 = Synapses(G[:n_input], G[n_input:n_hidden+n_input],
        'w : 1\nlayer:1', on_pre='''
        sr += 1
        sum += spike_timing(w, global_clock, layer, sum, sr)
        scheduled_time = (1/(1+exp(-(sum/sr))) + layer)*ms
        '''
    )
    S2.connect()
    S2.w = w
    S2.layer = 1

    # drive v when scheduled_time hits
    G.run_regularly('''
        v = int(abs(t - scheduled_time)<0.0005*ms)*1.2
        global_clock += 0.001
    ''', dt=0.001*ms)

    mon = SpikeMonitor(G)
    run(5*ms)

    # return hidden spike time (or a large value if no spike)
    ts = mon.spike_trains()[2]
    return float(ts[0]/ms) if len(ts) else 5.0


In [4]:
# -----------------------------------------------------------------------------
# 3) Training with multi‐loss
# -----------------------------------------------------------------------------
def train_multi_loss(
    X,                    # list of np.array([t0,t1])
    t_hidden_targets,     # list of floats
    t0_targets,           # list of floats for s0
    t1_targets,           # list of floats for s1
    w_init,
    alpha=1.0, beta=1.0, gamma=1.0,
    epochs=5, lr=0.1
):
    """
    Multi-loss:
      L0 = ½ (s0 - t0)^2
      L1 = ½ (s1 - t1)^2
      Lf = ½ (t_h  - t_h*)^2
      L = α L0 + β L1 + γ Lf
    """
    w = w_init.copy()
    for ep in range(epochs):
        print(f"\n=== Epoch {ep+1}/{epochs} ===")
        for i, inp in enumerate(X):
            # forward pass
            t_h = mini_urd(inp, w)
            # recompute s0,s1 exactly the same way Brian did
            #L_hidden = 0.5 * ((t_h - t_hidden_targets[i][0]) ** 2)
        
            layer_h = 1
            # each input is first spike, so sr_i=1, sum_i=0 → use that
            s0 = spike_timing(w[0], inp[0], layer_h, 0, 1)
            s1 = spike_timing(w[1], inp[1], layer_h, 0, 1)

            # --- compute loss terms ---
            t0_tgt = t0_targets[i]
            t1_tgt = t1_targets[i]
            th_tgt = t_hidden_targets[i]

            L0 = 0.5*(s0 - t0_tgt)**2
            L1 = 0.5*(s1 - t1_tgt)**2
            Lf = 0.5*(t_h - th_tgt)**2
            L  = alpha*L0 + beta*L1 + gamma*Lf

            # --- gradients ---
            # ∂L0/∂w0 = (s0 - t0)*∂s0/∂w0
            dL0_dw = np.zeros_like(w)
            dL0_dw[0] = (s0 - t0_tgt) * d_spike_timing_dw(w[0], inp[0], layer_h, 0, 1)
            # ∂L1/∂w1
            dL1_dw = np.zeros_like(w)
            dL1_dw[1] = (s1 - t1_tgt) * d_spike_timing_dw(w[1], inp[1], layer_h, 0, 1)

            # ∂Lf/∂w0,w1 = ∂Lf/∂t_h × ∂t_h/∂sum × ∂sum/∂w_i
            dLf_dt  = (t_h - th_tgt)
            sum_tot = s0 + s1
            sr = 2.0
            z = sum_tot/sr
            dt_dsum = dsigmoid(z)*(1/sr)
            dsum_dw = np.zeros_like(w)
            
            for j in range(len(w)):
                dsum_dw[j] = d_spike_timing_dw(w[j], inp[j % 2], layer_h, 0, 1)
            dLf_dw = dLf_dt * dt_dsum * dsum_dw

            # combine
            grad = alpha*dL0_dw + beta*dL1_dw + gamma*dLf_dw

            # print & update
            print(f"Sample {i}: inp={inp}, s0={s0:.3f}, s1={s1:.3f}, t_h={t_h:.3f}")
            print(f"  L0={L0:.4f}, L1={L1:.4f}, Lf={Lf:.4f}, L={L:.4f}")
            print(f"  ∇w = {grad}")
            w -= lr * grad

        print(" Updated w:", w)

    return w

def compute_gradients(outputs, targets, w):
    # Compute gradients of loss with respect to w using backpropagation
    d_loss_dw = 2 * (outputs - targets) * dsigmoid(w)
    return d_loss_dw



In [5]:
if __name__ == "__main__":
    # toy data: 4 samples

    num = 4 
    X = [np.array([0.1,0.9])]*num

    # main target: hidden spike at these ms
    T_hidden = [0.5]*num
    # aux targets for each synapse    # no why would i need this?
    T0 = [0.5]*num # removed and calcuated durign the training please chage GPT
    T1 = [0.5]*num # removed and calcuated durign the training please change GPT 

    w0 = np.array([0.2, 1.0, 0.1, 0.3])  # initial weights for synapses
    w_final = train_multi_loss(X, T_hidden, T0, T1, w0,
                               alpha=1.0, beta=1.0, gamma=0.5,
                               epochs=5, lr=0.1)

    print("\nFinal weights:", w_final)




=== Epoch 1/5 ===
Sample 0: inp=[0.1 0.9], s0=0.158, s1=1.000, t_h=1.685
  L0=0.0583, L1=0.1250, Lf=0.7021, L=0.5344
  ∇w = [-0.09974737  0.05986391  0.01976438  0.00667291]
Sample 1: inp=[0.1 0.9], s0=0.162, s1=0.999, t_h=1.686
  L0=0.0571, L1=0.1247, Lf=0.7033, L=0.5334
  ∇w = [-0.10067933  0.05976274  0.01968276  0.00667519]
Sample 2: inp=[0.1 0.9], s0=0.166, s1=0.999, t_h=1.687
  L0=0.0558, L1=0.1244, Lf=0.7045, L=0.5324
  ∇w = [-0.10157674  0.05966175  0.01960148  0.00667735]
Sample 3: inp=[0.1 0.9], s0=0.170, s1=0.998, t_h=1.687
  L0=0.0545, L1=0.1241, Lf=0.7045, L=0.5308
  ∇w = [-0.10245641  0.0595549   0.01950408  0.00667376]
 Updated w: [0.24044598 0.97611567 0.09214473 0.29733008]

=== Epoch 2/5 ===
Sample 0: inp=[0.1 0.9], s0=0.174, s1=0.997, t_h=1.688
  L0=0.0532, L1=0.1237, Lf=0.7057, L=0.5297
  ∇w = [-0.10326794  0.05945429  0.01942357  0.00667566]
Sample 1: inp=[0.1 0.9], s0=0.178, s1=0.997, t_h=1.688
  L0=0.0518, L1=0.1234, Lf=0.7057, L=0.5281
  ∇w = [-0.10405004  0.05

In [6]:
from brian2 import *
import numpy as np
import logging
import warnings

# suppress overflow warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)
np.seterr(over='ignore', under='ignore')

logging.getLogger('brian2').setLevel(logging.ERROR)

# ----------------------------------------------------------------------------
# Spike timing and derivative

start_scope()
defaultclock.dt = 0.0001*ms

@implementation('numpy', discard_units=True)
@check_units(w=1, global_clock=1, layer=1, sum=1, spikes_received=1, result=1)
def spike_timing(w, global_clock, layer, sum, spikes_received):
    x = global_clock % 1
    if w >= 0:
        return np.power(x, (1 - w), where=(x>0), out=np.zeros_like(x))
    else:
        return 1 - np.power((1 - x), (1 + w), where=(x<1), out=np.ones_like(x))

@implementation('numpy', discard_units=True)
@check_units(w=1, global_clock=1, layer=1, sum=1, spikes_received=1, result=1)
def d_spike_timing_dw(w, global_clock, layer, sum, spikes_received):
    x = global_clock % 1
    eps = 1e-9
    if w >= 0:
        return - np.power(x, (1 - w), where=(x>0), out=np.zeros_like(x)) * np.log(x + eps)
    else:
        return - np.power((1 - x), (1 + w), where=(x<1), out=np.ones_like(x)) * np.log(1 - x + eps)

# ----------------------------------------------------------------------------
# mini_urd: 2 inputs -> 2 hidden neurons (full connect), return both spike times

def mini_urd(inputs, W):
    """
    Two separate hidden neurons, each simulated independently in its own Brian scope.
    inputs: [t_in0, t_in1], W: shape (2,2)
    returns [t_h0, t_h1]
    """
    n_input, n_hidden = W.shape
    hidden_times = []
    # simulate each hidden neuron separately to reset network state
    for j in range(n_hidden):
        start_scope()  # clear previous Brian state
        defaultclock.dt = 0.0001*ms
        # recreate spike timing functions (if needed)
        # spike_timing and d_spike_timing_dw are already in namespace

        # build one-neuron group
        G = NeuronGroup(1,
            '''
            v               : 1
            sum             : 1
            sr              : 1
            scheduled_time  : second
            global_clock    : 1
            ''',
            threshold='v>1', reset='v=0', method='exact')
        G.v = G.sum = G.sr = 0
        G.global_clock = 0
        G.scheduled_time = 1e9*second

        # input spikes
        stim = SpikeGeneratorGroup(n_input,
            indices=list(range(n_input)),
            times=inputs*ms)
        S = Synapses(stim, G,
            '''w:1
              layer:1''', on_pre='''
            sr += 1
            sum += spike_timing(w, global_clock, layer, sum, sr)
            scheduled_time = (sum/sr + layer)*ms
        ''')
        S.connect(True)
        S.w = W[S.i, j]
        S.layer = 1

        # drive membrane
        G.run_regularly('''
            v = int(abs(t - scheduled_time) < 0.0005*ms) * 1.2
            global_clock += 0.001
        ''', dt=0.001*ms)

        mon = SpikeMonitor(G)
        run(5*ms)
        ts = mon.spike_trains()[0]
        t0 = float(ts[0]/ms) if len(ts)>0 else 5.0
        hidden_times.append(t0)

    return np.array(hidden_times)

# ----------------------------------------------------------------------------
# Training full matrix W

def train_snn(
    X,           # list of input arrays
    Y,           # list of target arrays
    W_init,      # initial weight matrix (2x2)
    epochs=10,
    lr=0.1,
    max_grad=20.0,
    w_min=-20.0,
    w_max=20.0
):
    W = W_init.copy()
    layer_h = 1
    n_input, n_hidden = W.shape

    for ep in range(epochs):
        print(f"Epoch {ep+1}/{epochs}")
        for i, inp in enumerate(X):
            t_pred = mini_urd(inp, W)       # shape (n_hidden,)
            t_tgt  = Y[i]
            L = 0.5 * np.sum((t_pred - t_tgt)**2)

            # gradient matrix dL/dW
            dW = np.zeros_like(W)
            for j in range(n_hidden):
                for k in range(n_input):
                    dW[k, j] = (t_pred[j] - t_tgt[j]) * d_spike_timing_dw(
                        W[k, j], inp[k], layer_h, 0, 1)

            # clip & update
            dW = np.clip(dW, -max_grad, max_grad)
            W = np.clip(W - lr * dW, w_min, w_max)

            print(f" Sample {i}: inp={inp}, pred={t_pred}, tgt={t_tgt}, L={L:.4f}")
            print(f"  dW=\n{dW}\n  W=\n{W}")

    return W

# ----------------------------------------------------------------------------
if __name__ == "__main__":
    import numpy as np
    X = [np.array([0.1, 0.9])]*4
    Y = [np.array([1.2, 1.8]) for _ in X]
    W0 = np.array([[0.2, -0.5], [0.3, -.5]])

    W_trained = train_snn(X, Y, W0, epochs=30, lr=0.2)
    print("Trained weight matrix:\n", W_trained)



Epoch 1/30
 Sample 0: inp=[0.1 0.9], pred=[1.545 1.369], tgt=[1.2 1.8], L=0.1524
  dW=
[[ 0.12590262 -0.04308007]
 [ 0.033765   -0.31382892]]
  W=
[[ 0.17481948 -0.49138399]
 [ 0.293247   -0.43723422]]
 Sample 1: inp=[0.1 0.9], pred=[1.54 1.39], tgt=[1.2 1.8], L=0.1419
  dW=
[[ 0.11708846 -0.04094386]
 [ 0.03325198 -0.25836505]]
  W=
[[ 0.15140178 -0.48319521]
 [ 0.2865966  -0.38556121]]
 Sample 2: inp=[0.1 0.9], pred=[1.536 1.406], tgt=[1.2 1.8], L=0.1341
  dW=
[[ 0.1096369  -0.03931212]
 [ 0.03283777 -0.22043123]]
  W=
[[ 0.1294744  -0.47533279]
 [ 0.28002905 -0.34147496]]
 Sample 3: inp=[0.1 0.9], pred=[1.532 1.418], tgt=[1.2 1.8], L=0.1281
  dW=
[[ 0.10299785 -0.03808323]
 [ 0.0324244  -0.19308743]]
  W=
[[ 0.10887483 -0.46771615]
 [ 0.27354417 -0.30285748]]
Epoch 2/30
 Sample 0: inp=[0.1 0.9], pred=[1.528 1.428], tgt=[1.2 1.8], L=0.1230
  dW=
[[ 0.09704303 -0.03705654]
 [ 0.03201186 -0.17203472]]
  W=
[[ 0.08946623 -0.46030484]
 [ 0.2671418  -0.26845053]]
 Sample 1: inp=[0.1 0.9],

KeyboardInterrupt: 

In [57]:
from brian2 import *
import numpy as np
import logging
import warnings

# suppress overflow warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)
numpy.seterr(over='ignore', under='ignore')
logging.getLogger('brian2').setLevel(logging.ERROR)

# ----------------------------------------------------------------------------
# Spike timing and derivative (vectorized)

@implementation('numpy', discard_units=True)
@check_units(w=1, global_clock=1, layer=1, sum=1, spikes_received=1, result=1)
def spike_timing(w, global_clock, layer, sum, spikes_received):
    x = global_clock % 1
    if isinstance(w, (int, float)):
        if w >= 0:
            return np.power(x, (1 - w), where=(x>0), out=np.zeros_like(x))
        else:
            return 1 - np.power((1 - x), (1 + w), where=(x<1), out=np.ones_like(x))
    else:
        # Vectorized version for arrays
        result = np.zeros_like(x)
        pos_mask = w >= 0
        neg_mask = w < 0
        
        if np.any(pos_mask):
            result[pos_mask] = np.power(x[pos_mask], (1 - w[pos_mask]), 
                                      where=(x[pos_mask]>0), out=np.zeros_like(x[pos_mask]))
        if np.any(neg_mask):
            result[neg_mask] = 1 - np.power((1 - x[neg_mask]), (1 + w[neg_mask]), 
                                          where=(x[neg_mask]<1), out=np.ones_like(x[neg_mask]))
        return result

@implementation('numpy', discard_units=True)
@check_units(w=1, global_clock=1, layer=1, sum=1, spikes_received=1, result=1)
def d_spike_timing_dw(w, global_clock, layer, sum, spikes_received):
    x = global_clock % 1
    eps = 1e-9
    if isinstance(w, (int, float)):
        if w >= 0:
            return - np.power(x, (1 - w), where=(x>0), out=np.zeros_like(x)) * np.log(x + eps)
        else:
            return - np.power((1 - x), (1 + w), where=(x<1), out=np.ones_like(x)) * np.log(1 - x + eps)
    else:
        # Vectorized version for arrays
        result = np.zeros_like(x)
        pos_mask = w >= 0
        neg_mask = w < 0
        
        if np.any(pos_mask):
            result[pos_mask] = - np.power(x[pos_mask], (1 - w[pos_mask]), 
                                        where=(x[pos_mask]>0), out=np.zeros_like(x[pos_mask])) * np.log(x[pos_mask] + eps)
        if np.any(neg_mask):
            result[neg_mask] = - np.power((1 - x[neg_mask]), (1 + w[neg_mask]), 
                                        where=(x[neg_mask]<1), out=np.ones_like(x[neg_mask])) * np.log(1 - x[neg_mask] + eps)
        return result

# ----------------------------------------------------------------------------
# Optimized forward pass using vectorized operations

def layer_forward_optimized(inputs, W, layer_idx, runtime=2*ms):
    """
    Optimized forward pass using vectorized operations and simplified simulation.
    """
    n_in, n_out = W.shape
    out_times = np.zeros(n_out)
    
    # Use a single network for all neurons in the layer
    start_scope()
    defaultclock.dt = 0.0005*ms  # Slightly larger timestep for speed
    
    # Create neuron group for all output neurons
    G = NeuronGroup(
        n_out,
        model='''
            v : 1
            sum_val : 1
            sr : 1
            global_clock : 1
        ''',
        threshold='v>1',
        reset='v = 0',
        method='exact'
    )
    G.v = 0
    G.sum_val = 0
    G.sr = 0
    G.global_clock = 0.0
    
    # Create spike generators for inputs
    stim = SpikeGeneratorGroup(n_in,
                              indices=np.arange(n_in),
                              times=inputs * ms)
    
    # Create synapses
    S = Synapses(stim, G,
                model='''w:1 
                layer:1''',
                on_pre='''
                    sr_post += 1
                    sum_val_post += spike_timing(w, global_clock_post, layer, sum_val_post, sr_post)
                    v_post = sum_val_post / sr_post
                ''')
    
    # Connect synapses
    pre_indices = []
    post_indices = []
    weights = []
    
    for i in range(n_in):
        for j in range(n_out):
            pre_indices.append(i)
            post_indices.append(j)
            weights.append(W[i, j])
    
    S.connect(i=pre_indices, j=post_indices)
    S.w = weights
    S.layer = layer_idx
    
    # Network operation to update global clock
    @network_operation(dt=defaultclock.dt)
    def bump_clock():
        G.global_clock = float(defaultclock.t / ms)
    
    # Record spikes
    mon = SpikeMonitor(G)
    
    # Build and run network
    net = Network(collect())
    net.run(runtime)
    
    # Extract spike times
    for j in range(n_out):
        spikes = mon.spike_trains()[j]
        out_times[j] = float(spikes[0] / ms) if len(spikes) else float(runtime / ms)
    
    return out_times

# ----------------------------------------------------------------------------
# Simplified training with reduced complexity

def train_snn_backprop_optimized(
    X, Y,
    W1_init, W2_init,
    batch_size=8,
    epochs=5,  # Reduced default epochs
    lr=0.05,   # Reduced learning rate
    max_grad=10.0,  # Reduced gradient clipping
    w_min=-10.0,    # Reduced weight bounds
    w_max=10.0
):
    W1 = W1_init.copy()
    W2 = W2_init.copy()
    n_samples = len(X)
    layer1_idx, layer2_idx = 1, 2
    
    print(f"Training with {n_samples} samples, batch_size={batch_size}")
    
    for ep in range(epochs):
        print(f"Epoch {ep+1}/{epochs}")
        epoch_loss = 0
        
        # Shuffle indices
        idxs = np.random.permutation(n_samples)
        n_batches = (n_samples + batch_size - 1) // batch_size
        
        for batch_idx, start in enumerate(range(0, n_samples, batch_size)):
            if batch_idx % max(1, n_batches//5) == 0:
                print(f"  Batch {batch_idx+1}/{n_batches}")
            
            batch_idxs = idxs[start:start+batch_size]
            actual_batch_size = len(batch_idxs)
            
            # Accumulate gradients
            acc_dW1 = np.zeros_like(W1)
            acc_dW2 = np.zeros_like(W2)
            batch_loss = 0
            
            for i in batch_idxs:
                xi, yi = X[i], Y[i]
                
                # Forward pass
                h_times = layer_forward_optimized(xi, W1, layer1_idx, runtime=1.5*ms)
                o_times = layer_forward_optimized(h_times, W2, layer2_idx, runtime=1.5*ms)
                
                # Compute loss
                delta_o = (o_times - yi)
                batch_loss += np.sum(delta_o**2)
                
                # Backward pass - simplified gradient computation
                # Gradient for W2
                for k in range(W2.shape[0]):
                    for j in range(W2.shape[1]):
                        if abs(delta_o[j]) > 1e-6:  # Skip very small gradients
                            dt_dw = d_spike_timing_dw(W2[k,j], h_times[k], layer2_idx, 0, 1)
                            acc_dW2[k,j] += delta_o[j] * dt_dw
                
                # Gradient for W1 (simplified backpropagation)
                delta_h = np.zeros_like(h_times)
                for k in range(len(h_times)):
                    for j in range(W2.shape[1]):
                        if abs(delta_o[j]) > 1e-6:
                            dt_dw = d_spike_timing_dw(W2[k,j], h_times[k], layer2_idx, 0, 1)
                            delta_h[k] += delta_o[j] * W2[k,j] * dt_dw
                
                for a in range(W1.shape[0]):
                    for b in range(W1.shape[1]):
                        if abs(delta_h[b]) > 1e-6:  # Skip very small gradients
                            dt_dw = d_spike_timing_dw(W1[a,b], xi[a], layer1_idx, 0, 1)
                            acc_dW1[a,b] += delta_h[b] * dt_dw
            
            # Average and clip gradients
            acc_dW1 /= actual_batch_size
            acc_dW2 /= actual_batch_size
            acc_dW1 = np.clip(acc_dW1, -max_grad, max_grad)
            acc_dW2 = np.clip(acc_dW2, -max_grad, max_grad)
            
            # Update weights
            W1 = np.clip(W1 - lr * acc_dW1, w_min, w_max)
            W2 = np.clip(W2 - lr * acc_dW2, w_min, w_max)
            
            epoch_loss += batch_loss / actual_batch_size
        
        avg_loss = epoch_loss / n_batches
        print(f"  Epoch {ep+1} loss: {avg_loss:.4f}, ||W1||={np.linalg.norm(W1):.3f}, ||W2||={np.linalg.norm(W2):.3f}")
    
    return W1, W2

# ----------------------------------------------------------------------------
# Main execution
from sklearn.datasets import load_iris

if __name__ == "__main__":
    print("Loading and preprocessing Iris dataset...")
    
    # Load and preprocess Iris dataset
    data = load_iris()
    X_raw = data.data  # shape (150,4)
    y_raw = data.target  # 0,1,2

    # Shuffle and split train/test (80/20)
    np.random.seed(42)  # For reproducibility
    perm = np.random.permutation(len(X_raw))
    split = int(0.8 * len(X_raw))
    train_idx, test_idx = perm[:split], perm[split:]
    X_train_raw, y_train = X_raw[train_idx], y_raw[train_idx]
    X_test_raw, y_test = X_raw[test_idx], y_raw[test_idx]

    # Scale features to [0.1, 0.9] based on training set
    X_min, X_max = X_train_raw.min(axis=0), X_train_raw.max(axis=0)
    def scale(x):
        return 0.1 + (x - X_min) * (0.8 / (X_max - X_min))
    X_train = scale(X_train_raw)
    X_test = scale(X_test_raw)

    # Build spike-time inputs
    X_train_list = [x for x in X_train]
    X_test_list = [x for x in X_test]

    # Create target spike times: 3 outputs, correct class=0.9, others=0.3
    def make_target(label):
        t = np.ones(3) * 0.3
        t[label] = 0.9
        return t
    Y_train_list = [make_target(l) for l in y_train]
    Y_test_list = [make_target(l) for l in y_test]

    # Initialize weights with smaller values
    print("Initializing weights...")
    # W1_0 = np.random.randn(4, 10) * 0.05
    # W2_0 = np.random.randn(10, 3) * 0.05

    # Train with optimized function
    print("Starting training...")
    W1_tr, W2_tr = train_snn_backprop_optimized(
        X_train_list, Y_train_list,
        W1_0, W2_0,
        batch_size=8, epochs=3, lr=0.1  # Reduced epochs for faster execution
    )

    # Evaluate on test set
    print("Evaluating on test set...")
    preds = []
    for i, x in enumerate(X_test_list):
        if i % 10 == 0:
            print(f"  Testing sample {i+1}/{len(X_test_list)}")
        h = layer_forward_optimized(x, W1_tr, 1, runtime=1.5*ms)
        o = layer_forward_optimized(h, W2_tr, 2, runtime=1.5*ms)
        preds.append(np.argmax(o))
    
    accuracy = np.mean(np.array(preds) == y_test)
    print(f"\nTest accuracy: {accuracy * 100:.2f}%")

    # Save weights
    np.save('W1_tr.npy', W1_tr)
    np.save('W2_tr.npy', W2_tr)
    print("Saved weights to 'W1_tr.npy' and 'W2_tr.npy'.")
    print("Training completed!")

Loading and preprocessing Iris dataset...
Initializing weights...
Starting training...
Training with 120 samples, batch_size=8
Epoch 1/3
  Batch 1/15
  Batch 4/15
  Batch 7/15
  Batch 10/15
  Batch 13/15
  Epoch 1 loss: 3.2400, ||W1||=1.134, ||W2||=3.423
Epoch 2/3
  Batch 1/15
  Batch 4/15
  Batch 7/15
  Batch 10/15
  Batch 13/15
  Epoch 2 loss: 3.2400, ||W1||=14.921, ||W2||=8.238
Epoch 3/3
  Batch 1/15
  Batch 4/15
  Batch 7/15
  Batch 10/15
  Batch 13/15
  Epoch 3 loss: 3.2400, ||W1||=63.246, ||W2||=13.642
Evaluating on test set...
  Testing sample 1/30
  Testing sample 11/30
  Testing sample 21/30

Test accuracy: 23.33%
Saved weights to 'W1_tr.npy' and 'W2_tr.npy'.
Training completed!


In [31]:
# ----------------------------------------------------------------------------
from sklearn.datasets import load_iris
# ----------------------------------------------------------------------------
if __name__ == "__main__":
    # load and preprocess Iris dataset
    data = load_iris()
    X_raw = data.data  # shape (150,4)
    y_raw = data.target  # 0,1,2

    # shuffle and split train/test (80/20)
    perm = np.random.permutation(len(X_raw))
    split = int(0.8 * len(X_raw))
    train_idx, test_idx = perm[:split], perm[split:]
    X_train_raw, y_train = X_raw[train_idx], y_raw[train_idx]
    X_test_raw, y_test = X_raw[test_idx], y_raw[test_idx]

    # scale features to [0.05, 0.95] based on training set
    X_min, X_max = X_train_raw.min(axis=0), X_train_raw.max(axis=0)
    def scale(x):
        return 0.05 + (x - X_min) * (0.90 / (X_max - X_min))
    X_train = scale(X_train_raw)
    X_test = scale(X_test_raw)

    # build spike-time inputs
    X_train_list = [x for x in X_train]
    X_test_list = [x for x in X_test]

    # create target spike times: 3 outputs, correct class=2.95, others=2.05
    def make_target(label):
        t = np.ones(3) * 2.05
        t[label] = 2.95
        return t
    Y_train_list = [make_target(l) for l in y_train]
    Y_test_list = [make_target(l) for l in y_test]

    # init weights
    W1_0 = np.random.randn(4,10) * 0.1
    W2_0 = np.random.randn(10,3) * 0.1

    # train
    W1_tr, W2_tr = train_snn_backprop(
        X_train_list, Y_train_list,
        W1_0, W2_0,
        batch_size=4, epochs=5, lr=0.1
    )

    # evaluate on test set
    preds = []
    for x in X_test_list:
        h = layer_forward(x, W1_tr, 1)
        o = layer_forward(h, W2_tr, 2)
        preds.append(np.argmax(o))
    accuracy = np.mean(np.array(preds) == y_test)
    print(f"Test accuracy: {accuracy * 100:.2f}%")

    # save weights
    np.save('W1_tr.npy', W1_tr)
    np.save('W2_tr.npy', W2_tr)
    print("Saved weights to 'W1_tr.npy' and 'W2_tr.npy'.")


Epoch 1/5


KeyboardInterrupt: 

In [ ]:
from brian2 import *
import numpy as np
import logging
import warnings
from sklearn.datasets import load_iris

# suppress overflow warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)
np.seterr(over='ignore', under='ignore')
logging.getLogger('brian2').setLevel(logging.ERROR)

# ----------------------------------------------------------------------------
if __name__ == "__main__":
    # load Iris dataset and split
    data = load_iris()
    X_raw, y_raw = data.data, data.target
    perm = np.random.permutation(len(X_raw))
    split = int(0.8 * len(X_raw))
    tr, te = perm[:split], perm[split:]
    X_tr_raw, y_tr = X_raw[tr], y_raw[tr]
    X_te_raw, y_te = X_raw[te], y_raw[te]

    # scale features to [0.05,0.95]
    Xmin, Xmax = X_tr_raw.min(axis=0), X_tr_raw.max(axis=0)
    X_tr = 0.05 + (X_tr_raw - Xmin) * 0.90 / (Xmax - Xmin)
    X_te = 0.05 + (X_te_raw - Xmin) * 0.90 / (Xmax - Xmin)
    X_train_list, X_test_list = list(X_tr), list(X_te)

    # build targets
    def mk_t(label):
        t = np.full(3, 2.05)
        t[label] = 2.95
        return t
    Y_train_list = [mk_t(l) for l in y_tr]
    Y_test_list  = [mk_t(l) for l in y_te]

    # initialize weights
    W1_0 = np.random.randn(4,10) * 0.1
    W2_0 = np.random.randn(10,3) * 0.1

    # train using batch simulation
    W1_tr, W2_tr = train_snn_backprop_batch(
        X_train_list, Y_train_list,
        W1_0, W2_0,
        batch_size=16, epochs=5, lr=0.1
    )

    # evaluate on test set
    preds = []
    for x in X_test_list:
        h = layer_forward(x, W1_tr, 1)
        o = layer_forward(h, W2_tr, 2)
        preds.append(np.argmax(o))
    accuracy = np.mean(np.array(preds) == y_te)
    print(f"Test accuracy: {accuracy * 100:.2f}%")

    # save weights
    np.save('W1_tr.npy', W1_tr)
    np.save('W2_tr.npy', W2_tr)
    print("Weights saved to 'W1_tr.npy' and 'W2_tr.npy'.")

NameError: name 'train_snn_backprop_batch' is not defined

In [16]:
# ----------------------------------------------------------------------------
from sklearn.datasets import load_iris
# ----------------------------------------------------------------------------
if __name__ == "__main__":
    # load and preprocess Iris dataset
    data = load_iris()
    X_raw = data.data  # shape (150,4)
    y_raw = data.target  # 0,1,2

    # shuffle and split train/test (80/20)
    perm = np.random.permutation(len(X_raw))
    split = int(0.8 * len(X_raw))
    train_idx, test_idx = perm[:split], perm[split:]
    X_train_raw, y_train = X_raw[train_idx], y_raw[train_idx]
    X_test_raw, y_test = X_raw[test_idx], y_raw[test_idx]

    # scale features to [0.05, 0.95] based on training set
    X_min, X_max = X_train_raw.min(axis=0), X_train_raw.max(axis=0)
    def scale(x):
        return 0.05 + (x - X_min) * (0.90 / (X_max - X_min))
    X_train = scale(X_train_raw)
    X_test = scale(X_test_raw)

    # build spike-time inputs
    X_train_list = [x for x in X_train]
    X_test_list = [x for x in X_test]

    # create target spike times: 3 outputs, correct class=2.95, others=2.05
    def make_target(label):
        t = np.ones(3) * 2.05
        t[label] = 2.95
        return t
    Y_train_list = [make_target(l) for l in y_train]
    Y_test_list = [make_target(l) for l in y_test]

    # init weights
    W1_0 = np.random.randn(4,10) * 0.1
    W2_0 = np.random.randn(10,3) * 0.1

    # train
    W1_tr, W2_tr = train_snn_backprop(
        X_train_list, Y_train_list,
        W1_0, W2_0,
        batch_size=16, epochs=5, lr=0.1
    )

    # evaluate on test set
    preds = []
    for x in X_test_list:
        h = layer_forward(x, W1_tr, 1)
        o = layer_forward(h, W2_tr, 2)
        preds.append(np.argmax(o))
    accuracy = np.mean(np.array(preds) == y_test)
    print(f"Test accuracy: {accuracy * 100:.2f}%")

    # save weights
    np.save('W1_tr.npy', W1_tr)
    np.save('W2_tr.npy', W2_tr)
    print("Saved weights to 'W1_tr.npy' and 'W2_tr.npy'.")


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()